# **IMPORT LIBRARY**

---



In [1]:
# --- GRUP 1: IMPORT LIBRARY DAN SETUP PATH ---

# === Cell 1: Verifikasi Versi Python ===
print("--- GRUP 1: SEL 1 ---")
!python --version
print("Verifikasi versi Python selesai.\n")

--- GRUP 1: SEL 1 ---
Python 3.11.12
Verifikasi versi Python selesai.



In [2]:
# === Cell 2: Instalasi Library yang Diperlukan ===
# (Sesuai daftar Anda, tanpa TensorFlow dan matplotlib)
print("--- GRUP 1: SEL 2 ---")
!pip install pandas numpy gensim Sastrawi nltk scikit-learn scipy
print("Instalasi library yang diperlukan (jika ada) selesai.\n")

--- GRUP 1: SEL 2 ---
Instalasi library yang diperlukan (jika ada) selesai.



In [3]:
# === Cell 3: Import Library ===
print("--- GRUP 1: SEL 3 ---")
# Library Python Standar
import math
import re
import os
import json
from collections import defaultdict
import pickle # Untuk menyimpan/memuat objek Python
import logging # Untuk logging
from datetime import datetime # Untuk timestamp
import warnings # Untuk menangani peringatan

# Library Eksternal Utama (sesuai daftar Anda)
import pandas as pd
import numpy as np
import gensim
from gensim.models import KeyedVectors, FastText # Tetap menggunakan FastText untuk training
from gensim.models.callbacks import CallbackAny2Vec

# Untuk Text Preprocessing
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
# import nltk # NLTK tidak diimpor jika hanya stopwords Sastrawi
# Jika NLTK stopwords juga diperlukan:
# nltk.download('stopwords', quiet=True)
# from nltk.corpus import stopwords as nltk_stopwords

# Untuk Sklearn Utilities
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity as sklearn_cosine_similarity # Digunakan di referensi LSTM Anda
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, mean_squared_error, mean_absolute_error, r2_score

# Untuk Statistik (Korelasi)
from scipy.stats import pearsonr, spearmanr

# Untuk Google Drive
from google.colab import drive

print("Import library selesai.\n")

# Konfigurasi dasar untuk logging (opsional, bisa disesuaikan)
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=UserWarning) # Bisa lebih spesifik jika perlu

--- GRUP 1: SEL 3 ---
Import library selesai.



In [5]:
# === Cell 4: Verifikasi Versi Library Utama ===
print("--- GRUP 1: SEL 4 ---")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")
print(f"Gensim version: {gensim.__version__}")

# Import pkg_resources to get package version
import pkg_resources
try:
    sastrawi_version = pkg_resources.get_distribution("Sastrawi").version
    print(f"Sastrawi version: {sastrawi_version}")
except pkg_resources.DistributionNotFound:
    print("Sastrawi version: Not found (package not installed correctly?)")
except Exception as e:
    print(f"Error getting Sastrawi version: {e}")


import sklearn
print(f"Scikit-learn version: {sklearn.__version__}")
import scipy
print(f"SciPy version: {scipy.__version__}")
print("Verifikasi versi library selesai.\n")

--- GRUP 1: SEL 4 ---
NumPy version: 1.26.4
Pandas version: 2.2.2
Gensim version: 4.3.3
Sastrawi version: 1.0.1
Scikit-learn version: 1.6.1
SciPy version: 1.13.1
Verifikasi versi library selesai.



In [6]:
# === Cell 5: Mount Google Drive dan Definisikan/Buat Path ===
print("--- GRUP 1: SEL 5 ---")
try:
    drive.mount('/content/drive', force_remount=True)
    print("Google Drive berhasil di-mount.")

    # Definisikan direktori dasar proyek (sesuai permintaan terakhir Anda)
    BASE_PROJECT_DIR = '/content/drive/MyDrive/Model/'

    # Definisikan path untuk setiap direktori utama sesuai struktur Anda
    DATASET_DIR = os.path.join(BASE_PROJECT_DIR, '01_Dataset/')
    PREPROCESSING_DIR = os.path.join(BASE_PROJECT_DIR, '02_Preprocessing_Artifacts/')
    EMBEDDING_DIR = os.path.join(BASE_PROJECT_DIR, '03_Word_Embeddings/')
    LSTM_MANUAL_MODEL_DIR = os.path.join(BASE_PROJECT_DIR, '04_LSTM_Model/') # Disesuaikan dengan nama folder Anda
    LOGS_DIR = os.path.join(BASE_PROJECT_DIR, '05_Training_Logs/')

    # Path spesifik untuk subfolder di dalam 03_Word_Embeddings
    PRETRAINED_EMBEDDING_DIR = os.path.join(EMBEDDING_DIR, 'Pretrained/')
    FINE_TUNED_EMBEDDING_DIR = os.path.join(EMBEDDING_DIR, 'Fine_Tuned/')
    EMBEDDING_BEST_KV_SUBDIR = os.path.join(FINE_TUNED_EMBEDDING_DIR, 'embedding_best_kv/')
    EMBEDDING_FINAL_MODEL_SUBDIR = os.path.join(FINE_TUNED_EMBEDDING_DIR, 'embedding_final_model/')

    # Path untuk file-file spesifik yang akan digunakan/dihasilkan
    # Dataset
    dataset_path_global = os.path.join(DATASET_DIR, 'Semua_Soal.json')
    input_soal_txt_path_global = os.path.join(DATASET_DIR, 'input_soal.txt')

    # Preprocessing Artifacts
    tokenizer_lstm_path_global = os.path.join(PREPROCESSING_DIR, 'tokenizer_lstm.pkl') # Meskipun LSTM manual mungkin tidak pakai ini

    # Embedding Pra-latih
    pretrained_embedding_path_global = os.path.join(PRETRAINED_EMBEDDING_DIR, 'cc.id.300.vec')

    # Embedding Fine-Tuned (akan dinamai "glove_" tapi dilatih dengan FastText)
    embedding_training_metadata_path_global = os.path.join(FINE_TUNED_EMBEDDING_DIR, 'embedding_training_metadata.json')
    embedding_best_kv_file_path_global = os.path.join(EMBEDDING_BEST_KV_SUBDIR, 'glove_best_similarity_keyedvectors.kv')
    # Path untuk model FastText utama yang akan kita anggap sebagai "glove_finetuned"
    glove_finetuned_model_path_global = os.path.join(EMBEDDING_FINAL_MODEL_SUBDIR, 'glove_finetuned.model')
    # Path untuk model FastText final yang mungkin disimpan oleh logger Gensim
    glove_final_model_internal_path_global = os.path.join(EMBEDDING_FINAL_MODEL_SUBDIR, 'glove_final_model.model')
    # File .npy akan dibuat oleh gensim saat menyimpan model FastText jika ada ngrams
    # glove_final_model_vectors_ngrams_path = os.path.join(EMBEDDING_FINAL_MODEL_SUBDIR, 'glove_final_model.model.wv.vectors_ngrams.npy')
    # glove_finetuned_vectors_ngrams_path = os.path.join(EMBEDDING_FINAL_MODEL_SUBDIR, 'glove_finetuned.model.wv.vectors_ngrams.npy')
    mean_fasttext_vector_file_path_global = os.path.join(FINE_TUNED_EMBEDDING_DIR, 'mean_fasttext_vector.npy') # Dipindah ke sini

    # Model LSTM (Manual NumPy) dan Metadatanya
    # Di *notebook* asli Anda, path LSTM adalah 'final_lstm_model.h5'. Karena kita pakai NumPy, kita ganti ke .npz
    # dan metadata .json-nya.
    lstm_manual_model_file_path_global = os.path.join(LSTM_MANUAL_MODEL_DIR, 'final_lstm_model.npz') # Sesuai referensi LSTM manual, tapi nama file dari Anda
    lstm_manual_metadata_file_path_global = os.path.join(LSTM_MANUAL_MODEL_DIR, 'model_metadata.json') # Sesuai struktur Anda

    # Log Training
    embedding_training_log_file_global = os.path.join(LOGS_DIR, 'training_log.json')
    lstm_training_log_file_global = os.path.join(LOGS_DIR, 'training_lstm_log.json') # Mungkin tidak relevan untuk LSTM manual
    hasil_evaluasi_akhir_file_global = os.path.join(LOGS_DIR, 'hasil_evaluasi_akhir.json')


    # List semua direktori yang perlu dibuat
    directories_to_create = [
        BASE_PROJECT_DIR, DATASET_DIR, PREPROCESSING_DIR,
        PRETRAINED_EMBEDDING_DIR, FINE_TUNED_EMBEDDING_DIR,
        EMBEDDING_BEST_KV_SUBDIR, EMBEDDING_FINAL_MODEL_SUBDIR,
        LSTM_MANUAL_MODEL_DIR, LOGS_DIR
    ]

    # Buat semua direktori jika belum ada
    for path_dir in directories_to_create:
        os.makedirs(path_dir, exist_ok=True)
        # print(f"Memastikan direktori ada atau telah dibuat: {path_dir}") # Komentari agar output tidak terlalu verbose

    print(f"\nBASE_PROJECT_DIR telah diatur ke: {BASE_PROJECT_DIR}")
    print("Path direktori dan file utama telah didefinisikan.")

    # Verifikasi opsional untuk file penting
    essential_files_check = {
        "Dataset Utama": dataset_path_global,
        "Input Soal Txt": input_soal_txt_path_global,
        "Embedding Pra-latih (cc.id.300.vec)": pretrained_embedding_path_global
    }
    for desc, file_path_check in essential_files_check.items():
        if not os.path.exists(file_path_check):
            print(f"PERINGATAN: File esensial '{desc}' TIDAK ditemukan di '{file_path_check}'. Pastikan file ada.")
        else:
            print(f"File esensial '{desc}' ditemukan di '{file_path_check}'.")

    print("\nSetup path selesai.\n")

    # Perintah untuk mengecek isi MyDrive dan BASE_PROJECT_DIR (opsional, untuk debugging)
    # print("\nIsi dari MyDrive Anda:")
    # !ls "/content/drive/MyDrive/"
    # print(f"\nIsi dari BASE_PROJECT_DIR ({BASE_PROJECT_DIR}):")
    # !ls "{BASE_PROJECT_DIR}"

except Exception as e:
    print(f"Error saat mounting drive atau setup path: {e}")

--- GRUP 1: SEL 5 ---
Mounted at /content/drive
Google Drive berhasil di-mount.

BASE_PROJECT_DIR telah diatur ke: /content/drive/MyDrive/Model/
Path direktori dan file utama telah didefinisikan.
File esensial 'Dataset Utama' ditemukan di '/content/drive/MyDrive/Model/01_Dataset/Semua_Soal.json'.
PERINGATAN: File esensial 'Input Soal Txt' TIDAK ditemukan di '/content/drive/MyDrive/Model/01_Dataset/input_soal.txt'. Pastikan file ada.
PERINGATAN: File esensial 'Embedding Pra-latih (cc.id.300.vec)' TIDAK ditemukan di '/content/drive/MyDrive/Model/03_Word_Embeddings/Pretrained/cc.id.300.vec'. Pastikan file ada.

Setup path selesai.



In [7]:
# === Cell 6: Inisialisasi Global Variabel dan Sastrawi ===
# (Bagian dari referensi Cell 5 LSTM manual Anda dan setup Sastrawi)
print("--- GRUP 1: SEL 6 ---")

# Variabel Global untuk model dan data (akan diisi nanti)
ft_model_global = None # Model FastText hasil fine-tuning (akan dinamai seolah GloVe)
mean_fasttext_vector_global = np.zeros(300) # Default, akan di-load atau dihitung dari ft_model_global
embedding_model_large_vocab_global = None # Untuk cc.id.300.vec (pra-latih besar)
mean_vector_for_large_vocab_global = np.zeros(300) # Untuk cc.id.300.vec
lstm_manual_model_instance_global = None # Instance dari kelas LSTM manual

# Inisialisasi Sastrawi (jika belum diinisialisasi atau ingin memastikan)
try:
    stemmer_factory = StemmerFactory()
    stemmer_global = stemmer_factory.create_stemmer()
    stopword_remover_factory = StopWordRemoverFactory()
    stopwords_sastrawi_global = stopword_remover_factory.get_stop_words()
    active_stopwords_list_global = stopwords_sastrawi_global # Bisa ditambahkan custom stopwords di sini jika perlu
    print("Inisialisasi Sastrawi stemmer dan stopword remover berhasil.")
    print(f"Menggunakan daftar stopwords dari Sastrawi ({len(active_stopwords_list_global)} kata).")
except Exception as e:
    print(f"Gagal menginisialisasi Sastrawi: {e}")
    stemmer_global = None
    active_stopwords_list_global = []

# Variabel lain dari referensi LSTM manual Anda (jika relevan secara global)
# LSTM_INPUT_DIM_GLOBAL = 300 # Akan ditentukan dari vector_size embedding
# LSTM_HIDDEN_DIM_GLOBAL = 128 # Bisa diatur sebagai parameter
# LSTM_OUTPUT_DIM_GLOBAL = 1
# LSTM_LEARNING_RATE_GLOBAL = 0.01

print("Inisialisasi variabel global dan Sastrawi selesai.\n")

--- GRUP 1: SEL 6 ---
Inisialisasi Sastrawi stemmer dan stopword remover berhasil.
Menggunakan daftar stopwords dari Sastrawi (126 kata).
Inisialisasi variabel global dan Sastrawi selesai.



# **EMBEDDING TEKS**

---



In [ ]:
# --- GRUP 2: EMBEDDING TEKS ---

# === Cell 1: Inisialisasi Fungsi Preprocessing dan Similarity ===
print("--- GRUP 2: SEL 1 ---")

# Pastikan stemmer_global dan active_stopwords_list_global sudah ada dari Grup 1
if 'stemmer_global' not in globals() or 'active_stopwords_list_global' not in globals():
    print("PERINGATAN: Objek Sastrawi (stemmer/stopwords) tidak ditemukan secara global. Inisialisasi ulang...")
    try:
        stemmer_factory = StemmerFactory()
        stemmer_global = stemmer_factory.create_stemmer()
        stopword_remover_factory = StopWordRemoverFactory()
        stopwords_sastrawi_global = stopword_remover_factory.get_stop_words()
        active_stopwords_list_global = stopwords_sastrawi_global
        print("Inisialisasi ulang Sastrawi berhasil.")
    except Exception as e:
        print(f"Gagal inisialisasi ulang Sastrawi: {e}")
        # Fallback jika Sastrawi gagal total
        stemmer_global = type('obj', (object,), {'stem': lambda x: x})() # Stemmer dummy
        active_stopwords_list_global = []


def preprocess_text(text_input):
    """Membersihkan dan memproses teks: lowercase, hapus non-alfanumerik, tokenisasi, hapus stopwords, stemming."""
    if not isinstance(text_input, str):
        return []
    text = text_input.lower()
    text = re.sub(r'[^a-z0-9\s]', '', text) # Hanya biarkan huruf, angka, dan spasi
    words = text.split()

    # Gunakan active_stopwords_list_global dan stemmer_global
    # Cek apakah stemmer_global None atau merupakan stemmer dummy
    if stemmer_global is None or not hasattr(stemmer_global, 'stem') or stemmer_global.stem.__qualname__ == 'type.stem': # Cek stemmer dummy
         # print("Peringatan: Stemmer tidak tersedia atau dummy, stemming dilewati.")
         words_processed = [word for word in words if word not in active_stopwords_list_global and word.strip()]
    else:
        words_processed = [stemmer_global.stem(word) for word in words if word not in active_stopwords_list_global and word.strip()]
    return words_processed

def calculate_cosine_similarity(vec1, vec2):
    """Menghitung cosine similarity antara dua vektor numpy."""
    if not isinstance(vec1, np.ndarray) or not isinstance(vec2, np.ndarray) or vec1.size == 0 or vec2.size == 0:
        return 0.0
    vec1 = vec1.flatten()
    vec2 = vec2.flatten()
    if vec1.shape != vec2.shape: # Pastikan bentuknya sama setelah flatten
        # print(f"Peringatan Cosine Sim: Bentuk vektor tidak cocok setelah flatten. vec1: {vec1.shape}, vec2: {vec2.shape}")
        return 0.0

    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    if norm_vec1 == 0 or norm_vec2 == 0:
        return 0.0
    similarity = dot_product / (norm_vec1 * norm_vec2)
    return similarity

def text_to_vector_for_similarity(text_input, keyed_vectors_model, default_vector_value=None):
    """
    Mengonversi teks menjadi vektor rata-rata menggunakan model KeyedVectors.
    Menggunakan default_vector_value jika tidak ada kata yang ditemukan atau model tidak valid.
    """
    # Default vector harus disediakan dan memiliki dimensi yang benar
    if default_vector_value is None:
        # print("Peringatan (text_to_vector_for_similarity): default_vector_value tidak disediakan.")
        if keyed_vectors_model is not None and hasattr(keyed_vectors_model, 'vector_size'):
            default_vector_value = np.zeros(keyed_vectors_model.vector_size)
        else: # Kasus darurat jika model juga tidak ada
            default_vector_value = np.zeros(300) # Asumsi dimensi default jika semua gagal

    if keyed_vectors_model is None or not hasattr(keyed_vectors_model, 'vector_size'):
        # print("  Peringatan (text_to_vector_for_similarity): Model KeyedVectors tidak valid atau tidak dimuat.")
        return default_vector_value.copy() if isinstance(default_vector_value, np.ndarray) else np.zeros_like(default_vector_value)


    if not isinstance(text_input, str) or not text_input.strip():
        return default_vector_value.copy()

    words = preprocess_text(text_input)
    word_vectors_list = []
    if words:
        for word in words:
            if word in keyed_vectors_model: # Untuk KeyedVectors, cek langsung
                word_vectors_list.append(keyed_vectors_model[word])
            elif hasattr(keyed_vectors_model, 'wv') and word in keyed_vectors_model.wv: # Untuk model FastText penuh
                word_vectors_list.append(keyed_vectors_model.wv[word])


    if not word_vectors_list:
        return default_vector_value.copy()

    return np.mean(word_vectors_list, axis=0)

print("Fungsi preprocessing dan similarity telah didefinisikan.\n")

In [ ]:
# === Cell 2: Memuat Dataset dan Mempersiapkan Korpus untuk Fine-Tuning Embedding ===
print("--- GRUP 2: SEL 2 ---")
list_of_all_texts_for_embedding = []
corpus_for_embedding_fine_tuning = []
dataset_embedding_list_g2 = [] # g2 untuk menandakan ini dari Grup 2

# Pastikan dataset_path_global sudah didefinisikan di Grup 1
if 'dataset_path_global' not in globals() or not os.path.exists(dataset_path_global):
    print(f"Error: File dataset '{dataset_path_global}' tidak ditemukan. Lewati persiapan korpus.")
else:
    try:
        with open(dataset_path_global, 'r', encoding='utf-8') as f:
            dataset_embedding_list_g2 = json.load(f)
        print(f"Dataset untuk embedding berhasil dimuat dari '{dataset_path_global}' ({len(dataset_embedding_list_g2)} baris).")

        print("Mempersiapkan korpus teks untuk fine-tuning embedding...")
        for item_data in dataset_embedding_list_g2:
            pertanyaan = item_data.get('pertanyaan', "")
            jawaban_siswa = item_data.get('jawaban_siswa', "")
            kata_kunci = item_data.get('kata_kunci', [])

            if isinstance(pertanyaan, str) and pertanyaan.strip():
                list_of_all_texts_for_embedding.append(pertanyaan)
            if isinstance(jawaban_siswa, str) and jawaban_siswa.strip():
                list_of_all_texts_for_embedding.append(jawaban_siswa)
            if isinstance(kata_kunci, list):
                for kw in kata_kunci:
                    if isinstance(kw, str) and kw.strip():
                        list_of_all_texts_for_embedding.append(kw)
            elif isinstance(kata_kunci, str) and kata_kunci.strip(): # Jika kata kunci adalah string tunggal
                 list_of_all_texts_for_embedding.append(kata_kunci)


        # Preprocess semua teks dalam korpus ini
        # Hasilnya akan menjadi list of list of tokens
        if list_of_all_texts_for_embedding:
            corpus_for_embedding_fine_tuning = [preprocess_text(text) for text in list_of_all_texts_for_embedding]
            corpus_for_embedding_fine_tuning = [s for s in corpus_for_embedding_fine_tuning if s] # Hapus list token yang kosong
        else:
            print("Tidak ada teks yang valid untuk membentuk korpus embedding.")


        if not corpus_for_embedding_fine_tuning:
            print("PERINGATAN: Korpus 'corpus_for_embedding_fine_tuning' kosong setelah preprocessing. Cek dataset Anda.")
        else:
            print(f"Total dokumen/teks dalam korpus yang sudah diproses untuk fine-tuning: {len(corpus_for_embedding_fine_tuning)}")
            if corpus_for_embedding_fine_tuning:
                print(f"Contoh item pertama dalam korpus (sudah di-preprocess): {corpus_for_embedding_fine_tuning[0][:10]}...") # Tampilkan 10 token pertama
    except Exception as e:
        print(f"Error saat memuat atau memproses dataset untuk embedding: {e}")

print("Persiapan korpus untuk embedding selesai.\n")

In [ ]:
# === Cell 3: Contoh Preprocessing Teks ===
print("--- GRUP 2: SEL 3 ---")
if list_of_all_texts_for_embedding:
    original_text_for_sample_g2 = list_of_all_texts_for_embedding[0]
    print(f"Contoh teks asli sebelum preprocessing: '{original_text_for_sample_g2}'")
    processed_sample_display_g2 = preprocess_text(original_text_for_sample_g2)
    print(f"Contoh teks setelah preprocessing (untuk display): {processed_sample_display_g2}")
else:
    print("Korpus kosong, tidak bisa menampilkan contoh preprocessing.")
print("Contoh preprocessing selesai ditampilkan.\n")

In [ ]:
# === Cell 4: Memuat Model Embedding Pra-latih Besar (cc.id.300.vec) ===
print("--- GRUP 2: SEL 4 ---")
# embedding_model_large_vocab_global dan mean_vector_for_large_vocab_global diinisialisasi di Grup 1

# Pastikan pretrained_embedding_path_global sudah ada
if 'pretrained_embedding_path_global' not in globals() or not os.path.exists(pretrained_embedding_path_global):
    print(f"Error: Path untuk embedding pra-latih '{pretrained_embedding_path_global}' tidak ditemukan atau variabel tidak ada.")
    embedding_model_large_vocab_global = None
    mean_vector_for_large_vocab_global = np.zeros(300) # Fallback
else:
    print(f"Mencoba memuat model embedding pra-latih besar dari {pretrained_embedding_path_global}...")
    try:
        embedding_model_large_vocab_global = KeyedVectors.load_word2vec_format(pretrained_embedding_path_global, binary=False)
        print("Model embedding pra-latih besar (cc.id.300.vec) berhasil dimuat.")
        print(f"  Ukuran Vocabulary: {len(embedding_model_large_vocab_global.key_to_index)} kata")
        print(f"  Dimensi Vektor: {embedding_model_large_vocab_global.vector_size}")
        if embedding_model_large_vocab_global.vectors.size > 0:
            mean_vector_for_large_vocab_global = np.mean(embedding_model_large_vocab_global.vectors, axis=0)
        else:
            # Fallback jika .vectors kosong (seharusnya tidak terjadi untuk model valid)
            mean_vector_for_large_vocab_global = np.zeros(embedding_model_large_vocab_global.vector_size)
        print("  Mean vector untuk model pra-latih besar telah dihitung.")

        # Contoh evaluasi kemiripan (opsional)
        sample_text_eval1_g2 = "raja adalah pemimpin kerajaan"
        sample_text_eval2_g2 = "ratu adalah pemimpin wanita di istana"
        vec_eval1_g2 = text_to_vector_for_similarity(sample_text_eval1_g2, embedding_model_large_vocab_global, mean_vector_for_large_vocab_global)
        vec_eval2_g2 = text_to_vector_for_similarity(sample_text_eval2_g2, embedding_model_large_vocab_global, mean_vector_for_large_vocab_global)
        similarity_score_eval_g2 = calculate_cosine_similarity(vec_eval1_g2, vec_eval2_g2)
        print(f"  Contoh similarity antara '{sample_text_eval1_g2}' dan '{sample_text_eval2_g2}': {similarity_score_eval_g2:.4f}")

    except Exception as e:
        print(f"Gagal memuat model embedding pra-latih besar: {e}")
        embedding_model_large_vocab_global = None
        mean_vector_for_large_vocab_global = np.zeros(300) # Fallback

if embedding_model_large_vocab_global is None:
    print("PERINGATAN: Model embedding pra-latih besar (cc.id.300.vec) gagal dimuat. Fungsi yang bergantung padanya mungkin tidak bekerja.")
print("Pemuatan model embedding pra-latih besar selesai.\n")

In [ ]:
# === Cell 5: Cek Keberadaan Model Embedding Fine-Tuned yang Sudah Ada ===
print("--- GRUP 2: SEL 5 ---")
skip_embedding_training = False
# ft_model_global dan mean_fasttext_vector_global diinisialisasi di Grup 1

# Path dari Grup 1
# glove_finetuned_model_path_global
# mean_fasttext_vector_file_path_global

if os.path.exists(glove_finetuned_model_path_global) and os.path.exists(mean_fasttext_vector_file_path_global):
    print(f"File model fine-tuned '{os.path.basename(glove_finetuned_model_path_global)}' dan mean vector ditemukan.")
    try:
        print(f"Mencoba memuat ft_model_global dari: {glove_finetuned_model_path_global}")
        ft_model_global = FastText.load(glove_finetuned_model_path_global)
        print(f"  ft_model_global (model '{os.path.basename(glove_finetuned_model_path_global)}') berhasil dimuat.")
        print(f"    Dimensi vektor: {ft_model_global.wv.vector_size}, Vocab: {len(ft_model_global.wv.key_to_index)}")

        print(f"Mencoba memuat mean_fasttext_vector_global dari: {mean_fasttext_vector_file_path_global}")
        mean_fasttext_vector_global = np.load(mean_fasttext_vector_file_path_global)
        print(f"  mean_fasttext_vector_global berhasil dimuat dengan shape: {mean_fasttext_vector_global.shape}")

        if ft_model_global.wv.vector_size != mean_fasttext_vector_global.shape[0]:
             print("PERINGATAN: Dimensi vektor ft_model_global tidak cocok dengan mean_fasttext_vector_global. Training mungkin diperlukan.")
             skip_embedding_training = False
        else:
            print("Model embedding fine-tuned dan mean vector berhasil dimuat. Training embedding akan dilewati.")
            skip_embedding_training = True
    except Exception as e:
        print(f"Gagal memuat model/mean vector yang sudah ada: {e}. Akan melanjutkan dengan training embedding.")
        skip_embedding_training = False
        ft_model_global = None # Reset jika gagal load
        mean_fasttext_vector_global = np.zeros(embedding_model_large_vocab_global.vector_size if embedding_model_large_vocab_global else 300)
else:
    print(f"File model fine-tuned '{os.path.basename(glove_finetuned_model_path_global)}' atau mean vector tidak ditemukan. Akan melanjutkan dengan training embedding.")
    skip_embedding_training = False
    ft_model_global = None
    mean_fasttext_vector_global = np.zeros(embedding_model_large_vocab_global.vector_size if embedding_model_large_vocab_global else 300)

print(f"Status skip_embedding_training: {skip_embedding_training}\n")

In [ ]:
# === Cell 6: Konfigurasi Parameter untuk Fine-Tuning Embedding ===
print("--- GRUP 2: SEL 6 ---")
if not skip_embedding_training:
    EPOCHS_EMBEDDING_G2 = 30       # Jumlah epoch untuk fine-tuning embedding (bisa disesuaikan)
    PATIENCE_EMBEDDING_G2 = 5      # Patience untuk early stopping manual (misalnya, jika similarity tidak meningkat)
    MIN_DELTA_EMBEDDING_G2 = 0.001 # Delta minimum untuk dianggap sebagai peningkatan similarity

    # Path dari Grup 1
    # embedding_training_log_file_global
    # embedding_training_metadata_path_global
    # EMBEDDING_BEST_KV_SUBDIR (digunakan oleh logger)
    # EMBEDDING_FINAL_MODEL_SUBDIR (digunakan oleh logger)

    print(f"Parameter fine-tuning embedding diatur: Epochs={EPOCHS_EMBEDDING_G2}, Patience={PATIENCE_EMBEDDING_G2}")
    print(f"Path log training embedding: {embedding_training_log_file_global}")
    print(f"Path metadata training embedding: {embedding_training_metadata_path_global}")
else:
    print("Training embedding dilewati, parameter tidak diatur.")
print("Konfigurasi parameter embedding selesai.\n")

In [ ]:
# === Cell 7: Kelas EmbeddingTrainingLogger (Callback Gensim) ===
print("--- GRUP 2: SEL 7 ---")

class EmbeddingTrainingLogger(CallbackAny2Vec):
    """Callback untuk logging selama training FastText dan menyimpan model terbaik."""
    def __init__(self, best_kv_dir, final_model_dir, log_file_path_param, vector_size_param,
                 eval_word1="fotosintesis", eval_word2="tumbuhan", patience=3, min_delta=0.001):
        self.epoch_manual = 0
        self.logs_data = {
            "start_time": datetime.now().isoformat(),
            "parameters": {
                "vector_size": vector_size_param,
                "eval_word1": eval_word1,
                "eval_word2": eval_word2,
                "patience_manual_early_stop": patience,
                "min_delta_manual_early_stop": min_delta
            },
            "early_stopping_triggered_manual": False,
            "epochs_detail": []
        }
        self.best_similarity_metric_val = -1.0 # Inisialisasi dengan nilai yang sangat rendah
        self.best_model_state_for_kv_val = None # Untuk menyimpan state KeyedVectors terbaik
        self.wait_manual_early_stop = 0
        self.patience_manual_early_stop = patience
        self.min_delta_manual_early_stop = min_delta

        self.best_kv_dir_path = best_kv_dir
        self.final_model_dir_path = final_model_dir
        self.log_file_path_val = log_file_path_param
        self.vector_size_val = vector_size_param
        self.eval_word1_val = stemmer_global.stem(eval_word1) # Stem kata evaluasi
        self.eval_word2_val = stemmer_global.stem(eval_word2) # Stem kata evaluasi

        os.makedirs(self.best_kv_dir_path, exist_ok=True)
        os.makedirs(self.final_model_dir_path, exist_ok=True)
        print(f"EmbeddingTrainingLogger diinisialisasi. Model terbaik KV akan disimpan di '{self.best_kv_dir_path}', model final di '{self.final_model_dir_path}'.")
        print(f"  Evaluasi similarity pada '{self.eval_word1_val}' dan '{self.eval_word2_val}'.")

    def log_manual_epoch_end(self, model, current_manual_epoch_num, early_stopping_status_flag=False):
        self.epoch_manual = current_manual_epoch_num
        current_similarity = 0.0

        # Pastikan kata ada di vocab sebelum menghitung similarity
        if self.eval_word1_val in model.wv.key_to_index and self.eval_word2_val in model.wv.key_to_index:
            current_similarity = model.wv.similarity(self.eval_word1_val, self.eval_word2_val)
        else:
            print(f"  Peringatan Logger: '{self.eval_word1_val}' atau '{self.eval_word2_val}' tidak ada di vocab model. Similarity akan 0.")

        epoch_log_data = {
            "epoch_manual": self.epoch_manual,
            "timestamp": datetime.now().isoformat(),
            "vocab_size": len(model.wv.key_to_index),
            f"similarity_sample_{self.eval_word1_val}_{self.eval_word2_val}": float(current_similarity),
            "early_stopping_triggered_this_epoch": early_stopping_status_flag
        }
        self.logs_data["epochs_detail"].append(epoch_log_data)
        self.logs_data["early_stopping_triggered_manual"] = early_stopping_status_flag

        # Simpan state KeyedVectors terbaik berdasarkan similarity
        if float(current_similarity) > self.best_similarity_metric_val + self.min_delta_manual_early_stop :
            self.best_similarity_metric_val = float(current_similarity)
            # Simpan seluruh model FastText saat ini sebagai kandidat terbaik untuk KV
            # karena KeyedVectors adalah bagian dari model FastText
            # Kita akan menyimpannya sebagai .kv di on_train_end jika ini tetap yang terbaik
            self.best_model_state_for_kv_val = model # Simpan referensi ke model, atau deepcopy jika khawatir termodifikasi
            print(f"    State KeyedVectors terbaik (dari model) diupdate pada epoch manual {self.epoch_manual} dengan similarity ({self.eval_word1_val}-{self.eval_word2_val}): {current_similarity:.4f}")
            self.wait_manual_early_stop = 0
        else:
            self.wait_manual_early_stop +=1
            if self.wait_manual_early_stop >= self.patience_manual_early_stop:
                 print(f"    Patience early stopping manual ({self.patience_manual_early_stop} epoch) tercapai.")
                 # Flag akan di-set oleh loop training utama

        try:
            with open(self.log_file_path_val, 'w') as f: json.dump(self.logs_data, f, indent=2)
        except Exception as e: print(f"    Gagal menyimpan log embedding: {str(e)}")

        print(f"  Epoch Embedding Manual {self.epoch_manual} | Similarity ({self.eval_word1_val}-{self.eval_word2_val}): {current_similarity:.4f}")
        if early_stopping_status_flag: print(f"    Early stopping manual terpicu pada epoch {self.epoch_manual}.")


    def on_train_end(self, model):
        """Dipanggil di akhir semua epoch fine-tuning."""
        self.logs_data["end_time"] = datetime.now().isoformat()
        self.logs_data["total_manual_epochs_trained"] = self.epoch_manual

        # Path dari Grup 1, dengan penamaan "glove_"
        # glove_final_model_internal_path_global
        # embedding_best_kv_file_path_global

        # Simpan model FastText final (state terakhir setelah semua epoch loop manual)
        # Ini BUKAN model utama yang akan kita sebut glove_finetuned.model, tapi log dari logger
        final_model_logger_path = os.path.join(self.final_model_dir_path, 'glove_final_model.model') # Sesuai struktur
        try:
            model.save(final_model_logger_path)
            print(f"Model FastText final (disimpan oleh logger) disimpan di: {final_model_logger_path}")
        except Exception as e: print(f"Gagal menyimpan model FastText final oleh logger: {e}")

        # Simpan KeyedVectors terbaik (jika ada peningkatan)
        if self.best_model_state_for_kv_val is not None:
            # best_model_state_for_kv_val adalah instance model FastText yang memberikan similarity terbaik
            # Kita simpan wv-nya sebagai KeyedVectors
            best_kv_path = os.path.join(self.best_kv_dir_path, 'glove_best_similarity_keyedvectors.kv') # Sesuai struktur
            try:
                self.best_model_state_for_kv_val.wv.save(best_kv_path)
                print(f"KeyedVectors terbaik (berdasarkan similarity) disimpan di: {best_kv_path}")
            except Exception as e: print(f"Error saat menyimpan KeyedVectors terbaik: {e}")
        else: print("Tidak ada state KeyedVectors terbaik yang disimpan (similarity tidak pernah meningkat atau model awal lebih baik).")

        try:
            with open(self.log_file_path_val, 'w') as f: json.dump(self.logs_data, f, indent=2)
            print(f"Log training embedding final disimpan di {self.log_file_path_val}")
        except Exception as e: print(f"Gagal menyimpan log embedding final: {str(e)}")

if not skip_embedding_training:
    print("Kelas EmbeddingTrainingLogger telah didefinisikan.")
else:
    print("Training embedding dilewati, kelas EmbeddingTrainingLogger tidak perlu didefinisikan ulang jika sudah ada.")
print("Definisi callback logger embedding selesai.\n")

In [ ]:
# === Cell 8: Inisialisasi Logger untuk Fine-Tuning Embedding ===
print("--- GRUP 2: SEL 8 ---")
embedding_ft_logger_g2 = None # Inisialisasi

if not skip_embedding_training:
    embedding_model_vector_size_g2 = 300 # Default
    if embedding_model_large_vocab_global is not None: # Gunakan dimensi dari model pra-latih besar
        embedding_model_vector_size_g2 = embedding_model_large_vocab_global.vector_size
        print(f"Menggunakan vector_size={embedding_model_vector_size_g2} dari model pra-latih besar untuk logger.")
    else:
        print(f"Peringatan: Model pra-latih besar tidak dimuat. Menggunakan vector_size default = {embedding_model_vector_size_g2} untuk logger.")


    # Path dari Grup 1
    # EMBEDDING_BEST_KV_SUBDIR, EMBEDDING_FINAL_MODEL_SUBDIR, embedding_training_log_file_global
    embedding_ft_logger_g2 = EmbeddingTrainingLogger(
        best_kv_dir=EMBEDDING_BEST_KV_SUBDIR,
        final_model_dir=EMBEDDING_FINAL_MODEL_SUBDIR,
        log_file_path_param=embedding_training_log_file_global,
        vector_size_param=embedding_model_vector_size_g2,
        patience=PATIENCE_EMBEDDING_G2, # Menggunakan parameter dari Cell 6
        min_delta=MIN_DELTA_EMBEDDING_G2 # Menggunakan parameter dari Cell 6
    )
    print("EmbeddingTrainingLogger diinisialisasi dan siap untuk fine-tuning.")
else:
    print("Training embedding dilewati, logger tidak diinisialisasi.")
print("Inisialisasi logger selesai.\n")

In [ ]:
# === Cell 9: Proses Fine-Tuning Embedding (Menggunakan FastText, dinamai seolah GloVe) ===
print("--- GRUP 2: SEL 9 ---")
# ft_model_global akan diisi di sini jika training berjalan

if not skip_embedding_training:
    if not corpus_for_embedding_fine_tuning:
        print("Error: Korpus 'corpus_for_embedding_fine_tuning' kosong. Training embedding tidak dapat dilanjutkan.")
    elif embedding_ft_logger_g2 is None:
        print("Error: Logger embedding (embedding_ft_logger_g2) tidak diinisialisasi. Training embedding tidak dapat dilanjutkan.")
    else:
        print(f"Korpus untuk training embedding ('corpus_for_embedding_fine_tuning') siap dengan {len(corpus_for_embedding_fine_tuning)} dokumen/kalimat.")
        if corpus_for_embedding_fine_tuning: print(f"  Contoh item pertama di korpus: {corpus_for_embedding_fine_tuning[0][:10]}...")

        # Parameter untuk model FastText baru (training dari awal)
        # Dimensi vektor diambil dari logger (yang mengambil dari model pra-latih besar)
        current_ft_vector_size = embedding_ft_logger_g2.vector_size_val
        GENSIM_INTERNAL_EPOCHS_PER_MANUAL_LOOP = 5 # Epoch internal Gensim per loop manual kita
        FT_WINDOW_SIZE = 5
        FT_MIN_COUNT = 2 # Abaikan kata dengan frekuensi di bawah ini
        FT_WORKERS = 4   # Jumlah worker threads
        FT_SG = 1        # 1 untuk skip-gram; 0 untuk CBOW
        FT_HS = 0        # 0 untuk negative sampling; 1 untuk hierarchical softmax
        FT_NEGATIVE = 10 # Jumlah noise words untuk negative sampling

        print(f"\nMenginisialisasi model FastText baru (training dari awal) dengan vector_size: {current_ft_vector_size}")
        temp_ft_model = FastText(
            vector_size=current_ft_vector_size,
            window=FT_WINDOW_SIZE,
            min_count=FT_MIN_COUNT,
            workers=FT_WORKERS,
            sg=FT_SG,
            hs=FT_HS,
            negative=FT_NEGATIVE,
            epochs=GENSIM_INTERNAL_EPOCHS_PER_MANUAL_LOOP, # Epoch internal Gensim
            callbacks=[embedding_ft_logger_g2] # Tambahkan callback internal Gensim jika perlu, tapi kita pakai loop manual
        )

        print("Membangun vocabulary model FastText dari korpus...")
        temp_ft_model.build_vocab(corpus_iterable=corpus_for_embedding_fine_tuning)
        print(f"  Ukuran vocabulary dari korpus: {len(temp_ft_model.wv.key_to_index)} kata")
        print(f"  Shape temp_ft_model.wv.vectors: {temp_ft_model.wv.vectors.shape}")

        # TIDAK menggunakan intersect_word2vec_format karena kita melatih dari awal
        # Jika ingin inisialisasi dengan pretrained besar:
        # if embedding_model_large_vocab_global is not None:
        #     print("Melakukan intersect_word2vec_format dengan model pra-latih besar...")
        #     temp_ft_model.intersect_word2vec_format(pretrained_embedding_path_global, binary=False, lockf=1.0)
        # else:
        #     print("Model pra-latih besar tidak tersedia untuk intersect, training dari scratch.")

        print(f"\nMemulai training embedding (maks {EPOCHS_EMBEDDING_G2} epoch manual, early stopping patience={PATIENCE_EMBEDDING_G2})...")
        actual_epochs_trained_embedding_g2 = 0

        for epoch_manual_num in range(1, EPOCHS_EMBEDDING_G2 + 1):
            actual_epochs_trained_embedding_g2 = epoch_manual_num
            print(f"\n--- Epoch Manual Training Embedding ke-{epoch_manual_num}/{EPOCHS_EMBEDDING_G2} ---")

            # Latih model FastText
            temp_ft_model.train(
                corpus_for_embedding_fine_tuning,
                total_examples=temp_ft_model.corpus_count,
                epochs=temp_ft_model.epochs # Menggunakan epochs yang diset saat inisialisasi model
            )

            # Logika Early Stopping Manual (dikontrol oleh logger sekarang)
            # Panggil log_manual_epoch_end dari logger, yang juga akan cek similarity
            # Dapatkan status early stopping dari logger
            embedding_ft_logger_g2.log_manual_epoch_end(temp_ft_model, epoch_manual_num,
                                                         embedding_ft_logger_g2.wait_manual_early_stop >= embedding_ft_logger_g2.patience_manual_early_stop)

            if embedding_ft_logger_g2.logs_data["early_stopping_triggered_manual"]:
                print(f"Early stopping manual terpicu setelah epoch {epoch_manual_num}. Menghentikan training embedding.")
                break
        # Akhir loop epoch manual

        embedding_ft_logger_g2.on_train_end(temp_ft_model) # Panggil on_train_end dari logger
        ft_model_global = temp_ft_model # Assign model yang sudah dilatih ke variabel global
        print("\n--- Training embedding (dari awal) selesai ---")

        # Hitung dan simpan mean_fasttext_vector_global
        if ft_model_global is not None and hasattr(ft_model_global, 'wv') and ft_model_global.wv.vectors.size > 0:
            mean_fasttext_vector_global = np.mean(ft_model_global.wv.vectors, axis=0)
            try:
                np.save(mean_fasttext_vector_file_path_global, mean_fasttext_vector_global)
                print(f"Mean vector dari ft_model_global berhasil disimpan di: {mean_fasttext_vector_file_path_global}")
            except Exception as e:
                print(f"Gagal menyimpan mean_fasttext_vector_global: {e}")
        else:
            print("ft_model_global tidak valid atau tidak memiliki vektor, mean_fasttext_vector_global tidak dihitung/disimpan.")
            # Fallback jika mean_fasttext_vector_global masih nol dari inisialisasi global
            if np.all(mean_fasttext_vector_global == 0):
                 mean_fasttext_vector_global = np.zeros(current_ft_vector_size if 'current_ft_vector_size' in locals() else 300)


else: # skip_embedding_training is True
    print("Training embedding dilewati karena model dan mean vector sudah ada dan dimuat.")
    # Pastikan ft_model_global dan mean_fasttext_vector_global sudah dimuat dengan benar di Cell 5

# Validasi akhir untuk ft_model_global dan mean_fasttext_vector_global
if ft_model_global is None:
    print("PERINGATAN KRITIS: ft_model_global tidak terdefinisi setelah Grup 2, Sel 9.")
else:
    print(f"ft_model_global siap digunakan (Vocab: {len(ft_model_global.wv.key_to_index) if hasattr(ft_model_global, 'wv') else 'N/A'}).")

if not isinstance(mean_fasttext_vector_global, np.ndarray) or mean_fasttext_vector_global.ndim != 1:
    print(f"PERINGATAN KRITIS: mean_fasttext_vector_global tidak valid setelah Grup 2, Sel 9. Shape: {mean_fasttext_vector_global.shape if isinstance(mean_fasttext_vector_global, np.ndarray) else type(mean_fasttext_vector_global)}")
else:
     print(f"mean_fasttext_vector_global siap digunakan (Shape: {mean_fasttext_vector_global.shape}).")

print("Proses fine-tuning embedding selesai.\n")

In [ ]:
# === Cell 10: Penyimpanan Akhir Model Embedding Utama dan Metadata ===
# (Sebelumnya Sel 11 di notebook asli Anda, disesuaikan)
print("--- GRUP 2: SEL 10 ---")

if ft_model_global is not None:
    # Simpan model FastText utama yang akan digunakan (hasil fine-tuning atau yang dimuat)
    # Ini akan menjadi model yang kita sebut "glove_finetuned.model"
    # Path dari Grup 1: glove_finetuned_model_path_global
    try:
        ft_model_global.save(glove_finetuned_model_path_global)
        print(f"Model FastText utama (glove_finetuned.model) berhasil disimpan/diperbarui di: {glove_finetuned_model_path_global}")
    except Exception as e:
        print(f"Gagal menyimpan model FastText utama '{glove_finetuned_model_path_global}': {e}")

    # Menyiapkan dan menyimpan metadata embedding
    # Path dari Grup 1: embedding_training_metadata_path_global

    # Dapatkan data dari logger jika training dilakukan, atau set default jika dilewati
    total_epochs_trained_val = 0
    early_stopping_triggered_val = False
    final_similarity_for_metadata_display_val = 0.0
    best_similarity_metric_for_metadata_val = 0.0
    eval_word1_md = "fotosintesis" # Kata default untuk display
    eval_word2_md = "tumbuhan"   # Kata default untuk display


    if not skip_embedding_training and embedding_ft_logger_g2 is not None: # Jika training dijalankan
        total_epochs_trained_val = embedding_ft_logger_g2.epoch_manual
        early_stopping_triggered_val = embedding_ft_logger_g2.logs_data["early_stopping_triggered_manual"]
        eval_word1_md = embedding_ft_logger_g2.eval_word1_val
        eval_word2_md = embedding_ft_logger_g2.eval_word2_val
        if eval_word1_md in ft_model_global.wv.key_to_index and eval_word2_md in ft_model_global.wv.key_to_index:
            final_similarity_for_metadata_display_val = ft_model_global.wv.similarity(eval_word1_md, eval_word2_md)
        best_similarity_metric_for_metadata_val = embedding_ft_logger_g2.best_similarity_metric_val
    elif ft_model_global is not None: # Jika training dilewati, coba hitung similarity dari model yang dimuat
        eval_word1_md_stemmed = stemmer_global.stem(eval_word1_md)
        eval_word2_md_stemmed = stemmer_global.stem(eval_word2_md)
        if eval_word1_md_stemmed in ft_model_global.wv.key_to_index and eval_word2_md_stemmed in ft_model_global.wv.key_to_index:
            final_similarity_for_metadata_display_val = ft_model_global.wv.similarity(eval_word1_md_stemmed, eval_word2_md_stemmed)
        # best_similarity_metric_for_metadata_val akan 0 jika training dilewati dan tidak ada data dari log lama
        eval_word1_md = eval_word1_md_stemmed
        eval_word2_md = eval_word2_md_stemmed


    # Hitung coverage pada dataset yang digunakan untuk fine-tuning
    coverage_train_val = 0.0
    if corpus_for_embedding_fine_tuning and ft_model_global:
        all_words_in_corpus_set = set(word for sentence in corpus_for_embedding_fine_tuning for word in sentence)
        words_in_vocab_and_corpus = [word for word in all_words_in_corpus_set if word in ft_model_global.wv.key_to_index]
        if all_words_in_corpus_set:
             coverage_train_val = len(words_in_vocab_and_corpus) / len(all_words_in_corpus_set)

    embedding_metadata_content_g2 = {
        'corpus_size_sentences_for_embedding': len(corpus_for_embedding_fine_tuning) if corpus_for_embedding_fine_tuning else 0,
        'vocab_size_trained_model': len(ft_model_global.wv.key_to_index) if ft_model_global and hasattr(ft_model_global, 'wv') else 0,
        'coverage_train_on_trained_model': float(f"{coverage_train_val:.4f}"), # Format sebagai float
        'total_epochs_trained_embedding': total_epochs_trained_val,
        'early_stopping_triggered_embedding_manual': early_stopping_triggered_val,
        f'final_similarity_sample ({eval_word1_md}-{eval_word2_md})': float(f"{final_similarity_for_metadata_display_val:.4f}"),
        f'best_similarity_sample ({eval_word1_md}-{eval_word2_md})': float(f"{best_similarity_metric_for_metadata_val:.4f}"),
        'embedding_model_type': 'FastText (trained from scratch, named as GloVe)',
        'embedding_vector_size': ft_model_global.vector_size if ft_model_global else (embedding_model_large_vocab_global.vector_size if embedding_model_large_vocab_global else 300),
        'embedding_window_size': ft_model_global.window if ft_model_global and hasattr(ft_model_global, 'window') else "N/A",
        'embedding_min_count': ft_model_global.min_count if ft_model_global and hasattr(ft_model_global, 'min_count') else "N/A",
        'embedding_sg_algorithm': ft_model_global.sg if ft_model_global and hasattr(ft_model_global, 'sg') else "N/A",
        'embedding_hs_algorithm': ft_model_global.hs if ft_model_global and hasattr(ft_model_global, 'hs') else "N/A",
        'embedding_negative_sampling': ft_model_global.negative if ft_model_global and hasattr(ft_model_global, 'negative') else "N/A",
        'embedding_training_date': datetime.now().isoformat()
    }
    try:
        with open(embedding_training_metadata_path_global, 'w') as f:
            json.dump(embedding_metadata_content_g2, f, indent=4)
        print(f"Metadata Training Embedding berhasil disimpan di: {embedding_training_metadata_path_global}")
    except Exception as e:
        print(f"Gagal menyimpan metadata training embedding: {e}")

    print("\n--- Rangkuman File yang Disimpan/Dimuat untuk Embedding ---")
    print(f"1. Model Embedding Utama (digunakan untuk LSTM & similarity): {glove_finetuned_model_path_global} {'(Dimuat)' if skip_embedding_training else '(Dilatih/Diperbarui)'}")
    print(f"2. Mean Vector dari Model Embedding Utama: {mean_fasttext_vector_file_path_global} {'(Dimuat)' if skip_embedding_training else '(Dihitung/Disimpan)'}")
    if os.path.exists(embedding_best_kv_file_path_global):
        print(f"3. KeyedVectors Terbaik (disimpan oleh logger jika training): {embedding_best_kv_file_path_global}")
    if os.path.exists(glove_final_model_internal_path_global):
         print(f"4. Model FastText Final (disimpan oleh logger jika training): {glove_final_model_internal_path_global}")
    print(f"5. Metadata Training Embedding: {embedding_training_metadata_path_global}")
    if os.path.exists(embedding_training_log_file_global):
        print(f"6. Log Training Embedding: {embedding_training_log_file_global}")

else: # ft_model_global is None
    print("Model embedding fine-tuned (ft_model_global) tidak tersedia. Penyimpanan metadata dilewati.")

print("Penyimpanan akhir model embedding dan metadata selesai.\n")

# **LSTM**

---



In [ ]:
# --- GRUP 3: IMPLEMENTASI LSTM MANUAL (NumPy) ---

# === Cell 1: Definisi Kelas LSTM Manual dan Fungsi Pendukung ===
print("--- GRUP 3: SEL 1 ---")

# Pastikan variabel path global sudah ada dari Grup 1
# lstm_manual_model_file_path_global
# lstm_manual_metadata_file_path_global
# dataset_path_global
# mean_fasttext_vector_file_path_global
# ft_model_global (diharapkan sudah dimuat atau dilatih di Grup 2)

if 'ft_model_global' not in globals() or ft_model_global is None:
    print("PERINGATAN: ft_model_global (model embedding fine-tuned) tidak ditemukan. LSTM mungkin tidak dapat dilatih atau dievaluasi dengan benar.")
    # Inisialisasi dummy ft_model jika tidak ada, agar tidak error, tapi ini akan menghasilkan vektor nol
    if 'embedding_model_large_vocab_global' in globals() and embedding_model_large_vocab_global is not None:
        print("Menggunakan embedding_model_large_vocab_global sebagai fallback untuk dimensi ft_model_global.")
        ft_model_global = embedding_model_large_vocab_global # Fallback sementara, idealnya ft_model_global ada
    else:
        # Jika semua model embedding tidak ada, buat dummy KeyedVectors
        dummy_vectors = KeyedVectors(vector_size=300)
        # Tambahkan satu kata dummy agar tidak error saat akses wv
        dummy_vectors.add_vectors(["<dummy>"], [np.zeros(300)])
        ft_model_global = type('obj', (object,), {'wv': dummy_vectors, 'vector_size': 300})() # Dummy model FastText
        print("Membuat dummy ft_model_global karena tidak ada model embedding yang dimuat.")


if 'mean_fasttext_vector_global' not in globals() or not isinstance(mean_fasttext_vector_global, np.ndarray) or mean_fasttext_vector_global.shape[0] == 0 :
    print(f"PERINGATAN: mean_fasttext_vector_global tidak valid. Mencoba memuat dari path: {mean_fasttext_vector_file_path_global}")
    try:
        mean_fasttext_vector_global = np.load(mean_fasttext_vector_file_path_global)
        if not isinstance(mean_fasttext_vector_global, np.ndarray) or mean_fasttext_vector_global.shape[0] == 0: # Cek lagi setelah load
            print(f"  Gagal memuat mean_fasttext_vector_global yang valid. Menggunakan vektor nol (dimensi 300).")
            mean_fasttext_vector_global = np.zeros(ft_model_global.wv.vector_size if hasattr(ft_model_global, 'wv') else 300)
        else:
            print(f"  mean_fasttext_vector_global berhasil dimuat dari file dengan shape {mean_fasttext_vector_global.shape}")
    except Exception as e:
        print(f"  Gagal memuat mean_fasttext_vector_global dari file: {e}. Menggunakan vektor nol (dimensi 300).")
        mean_fasttext_vector_global = np.zeros(ft_model_global.wv.vector_size if hasattr(ft_model_global, 'wv') else 300)


def save_lstm_manual_metadata(path, metadata):
    try:
        with open(path, 'w') as f: json.dump(metadata, f, indent=4)
        print(f"Metadata LSTM manual disimpan di: {path}")
    except Exception as e:
        print(f"Gagal menyimpan metadata LSTM manual: {e}")

def load_lstm_manual_metadata(path):
    try:
        with open(path, 'r') as f: return json.load(f)
    except FileNotFoundError:
        # print(f"File metadata LSTM manual tidak ditemukan di: {path}")
        return None
    except Exception as e:
        print(f"Gagal memuat metadata LSTM manual dari {path}: {e}")
        return None

def xavier_init(size_param, use_fan_in_fan_out=False):
    # Implementasi inisialisasi Xavier yang lebih umum
    if not isinstance(size_param, tuple) or len(size_param) == 0:
        raise ValueError("Parameter 'size_param' harus berupa tuple yang tidak kosong.")

    if len(size_param) == 1: # Untuk bias atau input layer tunggal
        fan_in = size_param[0]
        fan_out = 1 # Asumsi
    else: # Untuk weight matrix
        fan_in = size_param[1]  # Jumlah neuron di layer sebelumnya
        fan_out = size_param[0] # Jumlah neuron di layer saat ini

    if fan_in == 0: return np.zeros(size_param) # Hindari pembagian dengan nol

    if use_fan_in_fan_out: # Xavier/Glorot uniform dengan fan_in dan fan_out
        limit = np.sqrt(6.0 / (fan_in + fan_out))
        return np.random.uniform(-limit, limit, size=size_param)
    else: # Implementasi asli Anda (mirip He/Lecun normal)
        return np.random.randn(*size_param) * np.sqrt(1.0 / fan_in)


class LSTMManual:
    def __init__(self, input_dim, hidden_dim, output_dim, learning_rate=0.01):
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.learning_rate = learning_rate

        # Inisialisasi bobot dan bias
        # Matriks bobot untuk input dan hidden state sebelumnya
        # Ukuran: (hidden_dim, hidden_dim + input_dim)
        self.Wf = xavier_init((hidden_dim, hidden_dim + input_dim)) # Forget gate
        self.Wi = xavier_init((hidden_dim, hidden_dim + input_dim)) # Input gate
        self.Wo = xavier_init((hidden_dim, hidden_dim + input_dim)) # Output gate
        self.Wc = xavier_init((hidden_dim, hidden_dim + input_dim)) # Cell state candidate

        # Bias untuk setiap gate
        # Ukuran: (hidden_dim, 1)
        self.bf = np.zeros((hidden_dim, 1))
        self.bi = np.zeros((hidden_dim, 1))
        self.bo = np.zeros((hidden_dim, 1))
        self.bc = np.zeros((hidden_dim, 1))

        # Bobot dan bias untuk output layer
        # Ukuran Wy: (output_dim, hidden_dim)
        # Ukuran by: (output_dim, 1)
        self.Wy = xavier_init((output_dim, hidden_dim))
        self.by = np.zeros((output_dim, 1))

        self.cache = {} # Untuk menyimpan nilai intermediate selama forward pass

    def sigmoid(self, x_val):
        return 1 / (1 + np.exp(-np.clip(x_val, -500, 500))) # Clip untuk stabilitas numerik

    def tanh(self, x_val):
        return np.tanh(np.clip(x_val, -500, 500)) # Clip untuk stabilitas numerik

    def forward(self, x_sequence_input):
        # x_sequence_input adalah list dari vektor input, setiap vektor mewakili satu timestep.
        # Untuk kasus kita, x_sequence_input akan berisi SATU vektor (rata-rata embedding jawaban)
        # sehingga T (jumlah timestep) akan menjadi 1.

        T = len(x_sequence_input)
        if T == 0:
            # print("Peringatan Forward: x_sequence_input kosong.")
            return np.zeros((self.output_dim, 1)) # Kembalikan output default jika sekuens kosong

        # Inisialisasi hidden state dan cell state awal
        h_prev_val = np.zeros((self.hidden_dim, 1))
        c_prev_val = np.zeros((self.hidden_dim, 1))

        # Simpan nilai awal ke cache
        self.cache['h'] = {0: h_prev_val}
        self.cache['c'] = {0: c_prev_val}
        self.cache['concat'] = {}
        self.cache['f'] = {}
        self.cache['i'] = {}
        self.cache['o'] = {}
        self.cache['c_tilde'] = {}
        self.cache['x_sequence'] = x_sequence_input # Simpan sekuens input asli

        for t_step in range(T):
            # xt_val adalah vektor input pada timestep t
            xt_val = x_sequence_input[t_step].reshape(self.input_dim, 1)

            # Gabungkan hidden state sebelumnya (h_prev) dengan input saat ini (xt)
            concat_val = np.vstack((self.cache['h'][t_step], xt_val))
            self.cache['concat'][t_step + 1] = concat_val

            # Hitung nilai gate
            ft_val = self.sigmoid(np.dot(self.Wf, concat_val) + self.bf)  # Forget gate
            it_val = self.sigmoid(np.dot(self.Wi, concat_val) + self.bi)  # Input gate
            c_tilde_t_val = self.tanh(np.dot(self.Wc, concat_val) + self.bc) # Candidate cell state
            ot_val = self.sigmoid(np.dot(self.Wo, concat_val) + self.bo)  # Output gate

            # Hitung cell state baru dan hidden state baru
            ct_val = ft_val * self.cache['c'][t_step] + it_val * c_tilde_t_val # Cell state
            ht_val = ot_val * self.tanh(ct_val) # Hidden state

            # Simpan nilai intermediate ke cache untuk backward pass
            self.cache['h'][t_step + 1] = ht_val
            self.cache['c'][t_step + 1] = ct_val
            self.cache['f'][t_step + 1] = ft_val
            self.cache['i'][t_step + 1] = it_val
            self.cache['o'][t_step + 1] = ot_val
            self.cache['c_tilde'][t_step + 1] = c_tilde_t_val

        # Prediksi output menggunakan hidden state terakhir
        y_pred_val = np.dot(self.Wy, self.cache['h'][T]) + self.by
        # Karena kita memprediksi skor (nilai tunggal antara 0 dan 1), kita bisa tambahkan sigmoid di sini
        # Jika output_dim = 1 dan ini adalah regresi ke skor 0-1, sigmoid mungkin cocok.
        if self.output_dim == 1:
             y_pred_val = self.sigmoid(y_pred_val) # Asumsi skor akhir adalah probabilitas/nilai 0-1

        return y_pred_val

    def backward(self, dy_pred_input):
        T = len(self.cache['x_sequence'])
        if T == 0: return # Tidak ada yang di-backward jika tidak ada forward pass

        # Inisialisasi gradien
        dWf_val, dWi_val, dWo_val, dWc_val = np.zeros_like(self.Wf), np.zeros_like(self.Wi), np.zeros_like(self.Wo), np.zeros_like(self.Wc)
        dbf_val, dbi_val, dbo_val, dbc_val = np.zeros_like(self.bf), np.zeros_like(self.bi), np.zeros_like(self.bo), np.zeros_like(self.bc)

        # Gradien untuk output layer
        dWy_val = np.dot(dy_pred_input, self.cache['h'][T].T)
        dby_val = dy_pred_input

        # Inisialisasi gradien untuk hidden state dan cell state yang akan di-propagate
        dh_next_val = np.dot(self.Wy.T, dy_pred_input)
        dc_next_val = np.zeros_like(dh_next_val) # Gradien cell state dari timestep selanjutnya (awal: 0)

        # Loop backward melalui waktu
        for t_step in reversed(range(1, T + 1)):
            # Gradien untuk output gate (ot)
            dot_val = dh_next_val * self.tanh(self.cache['c'][t_step]) * self.cache['o'][t_step] * (1 - self.cache['o'][t_step])
            dWo_val += np.dot(dot_val, self.cache['concat'][t_step].T)
            dbo_val += dot_val

            # Gradien untuk cell state (ct)
            dc_t_val = dh_next_val * self.cache['o'][t_step] * (1 - self.tanh(self.cache['c'][t_step])**2) + dc_next_val

            # Gradien untuk candidate cell state (c_tilde_t)
            dc_tilde_t_val = dc_t_val * self.cache['i'][t_step] * (1 - self.cache['c_tilde'][t_step]**2)
            dWc_val += np.dot(dc_tilde_t_val, self.cache['concat'][t_step].T)
            dbc_val += dc_tilde_t_val

            # Gradien untuk input gate (it)
            dit_val = dc_t_val * self.cache['c_tilde'][t_step] * self.cache['i'][t_step] * (1 - self.cache['i'][t_step])
            dWi_val += np.dot(dit_val, self.cache['concat'][t_step].T)
            dbi_val += dit_val

            # Gradien untuk forget gate (ft)
            c_prev_t_minus_1_val = self.cache['c'][t_step - 1] # c_prev pada timestep t adalah c pada t-1
            dft_val = dc_t_val * c_prev_t_minus_1_val * self.cache['f'][t_step] * (1 - self.cache['f'][t_step])
            dWf_val += np.dot(dft_val, self.cache['concat'][t_step].T)
            dbf_val += dft_val

            # Gradien untuk concatenated input (h_prev_val + xt_val)
            d_concat_val = (np.dot(self.Wf.T, dft_val) +
                            np.dot(self.Wi.T, dit_val) +
                            np.dot(self.Wc.T, dc_tilde_t_val) +
                            np.dot(self.Wo.T, dot_val))

            # Pisahkan gradien untuk h_prev_val dan xt_val
            dh_next_val = d_concat_val[:self.hidden_dim, :] # Gradien untuk hidden state sebelumnya
            # dxt_val = d_concat_val[self.hidden_dim:, :] # Gradien untuk input (tidak digunakan untuk update bobot LSTM)

            # Update gradien cell state untuk propagasi ke timestep sebelumnya
            dc_next_val = dc_t_val * self.cache['f'][t_step]

        # Update bobot dan bias (Vanilla Gradient Descent)
        self.Wf -= self.learning_rate * np.clip(dWf_val, -1, 1) # Gradient clipping
        self.Wi -= self.learning_rate * np.clip(dWi_val, -1, 1)
        self.Wo -= self.learning_rate * np.clip(dWo_val, -1, 1)
        self.Wc -= self.learning_rate * np.clip(dWc_val, -1, 1)

        self.bf -= self.learning_rate * np.clip(dbf_val, -1, 1)
        self.bi -= self.learning_rate * np.clip(dbi_val, -1, 1)
        self.bo -= self.learning_rate * np.clip(dbo_val, -1, 1)
        self.bc -= self.learning_rate * np.clip(dbc_val, -1, 1)

        self.Wy -= self.learning_rate * np.clip(dWy_val, -1, 1)
        self.by -= self.learning_rate * np.clip(dby_val, -1, 1)


    def train(self, X_train_data, Y_train_data, epochs=10, batch_size=32): # Tambahkan batch_size
        num_samples = X_train_data.shape[0]
        if num_samples == 0:
            print("PERINGATAN: Data training kosong. Training LSTM dihentikan.")
            return

        # Asumsi Y_train_data adalah (num_samples, 1) atau (num_samples,)
        if Y_train_data.ndim == 1:
            Y_train_data = Y_train_data.reshape(-1, 1)

        for epoch_num in range(epochs):
            total_loss_epoch = 0
            permutation_indices = np.random.permutation(num_samples)
            X_train_shuffled_epoch = X_train_data[permutation_indices]
            Y_train_shuffled_epoch = Y_train_data[permutation_indices]

            for i in range(0, num_samples, batch_size):
                X_batch = X_train_shuffled_epoch[i:i+batch_size]
                Y_batch = Y_train_shuffled_epoch[i:i+batch_size]

                # Akumulasi gradien untuk batch
                dWf_batch, dWi_batch, dWo_batch, dWc_batch = np.zeros_like(self.Wf), np.zeros_like(self.Wi), np.zeros_like(self.Wo), np.zeros_like(self.Wc)
                dbf_batch, dbi_batch, dbo_batch, dbc_batch = np.zeros_like(self.bf), np.zeros_like(self.bi), np.zeros_like(self.bo), np.zeros_like(self.bc)
                dWy_batch, dby_batch = np.zeros_like(self.Wy), np.zeros_like(self.by)

                batch_loss = 0
                for j in range(X_batch.shape[0]):
                    x_sample_item = X_batch[j] # Ini adalah satu vektor (misal, rata-rata embedding)
                    y_true_sample_item = Y_batch[j].reshape(self.output_dim, 1)

                    # Input ke LSTM adalah sekuens, meskipun hanya 1 timestep (vektor rata-rata)
                    x_sequence_item = [x_sample_item]

                    y_pred_item = self.forward(x_sequence_item)
                    loss_item = 0.5 * np.sum((y_pred_item - y_true_sample_item)**2) # MSE Loss
                    batch_loss += loss_item

                    # Hitung gradien untuk sampel ini (dy_pred untuk MSE)
                    # Jika output layer adalah sigmoid, dy_pred perlu penyesuaian (turunan sigmoid * (y_pred - y_true))
                    # dy_pred_item = (y_pred_item - y_true_sample_item) # Gradien untuk MSE jika output linear
                    # Jika y_pred_item adalah hasil sigmoid(z_output), maka d(Loss)/d(z_output) = (y_pred - y_true) * y_pred * (1 - y_pred)
                    # Untuk kesederhanaan, kita asumsikan gradien yang masuk ke backward adalah d(Loss)/d(y_pred_final)
                    dy_pred_item = (y_pred_item - y_true_sample_item) * y_pred_item * (1 - y_pred_item) # Jika output sigmoid & MSE

                    # Panggil backward untuk menghitung gradien internal dan simpan ke cache sementara
                    # Modifikasi backward agar mengembalikan gradien, bukan langsung update
                    # Atau, backward mengakumulasi gradien untuk batch
                    # Untuk sekarang, backward yang ada mengupdate bobot, jadi kita panggil saja
                    # Ini akan menjadi Stochastic Gradient Descent jika batch_size=1, atau Mini-batch jika backward dimodif
                    # Dengan backward yang ada, ini efektif SGD jika dipanggil per sampel.
                    # Agar menjadi mini-batch, backward perlu diubah untuk mengakumulasi gradien

                    # Untuk implementasi mini-batch sederhana tanpa mengubah backward drastis:
                    # Hitung gradien per sampel dan akumulasi, lalu update setelah batch.
                    # Ini membutuhkan backward untuk *return* gradien, bukan apply.
                    # Untuk saat ini, kita biarkan backward mengupdate per sampel (SGD)
                    self.backward(dy_pred_item) # Ini akan mengupdate bobot per sampel

                total_loss_epoch += batch_loss / X_batch.shape[0] # Rata-rata loss per batch

            avg_loss_epoch = total_loss_epoch / (num_samples / batch_size) # Rata-rata loss per epoch
            if (epoch_num + 1) % 1 == 0: # Cetak setiap epoch
                 print(f"Epoch {epoch_num+1}/{epochs}, Average Loss: {avg_loss_epoch:.6f}")


    def save_model(self, path_param):
        try:
            np.savez(path_param,
                     Wf=self.Wf, Wi=self.Wi, Wo=self.Wo, Wc=self.Wc,
                     bf=self.bf, bi=self.bi, bo=self.bo, bc=self.bc,
                     Wy=self.Wy, by=self.by,
                     input_dim=self.input_dim, hidden_dim=self.hidden_dim,
                     output_dim=self.output_dim, learning_rate=self.learning_rate)
            print(f"Model LSTM manual disimpan di: {path_param}")
        except Exception as e:
            print(f"Gagal menyimpan model LSTM manual: {e}")

    def load_model(self, path_param):
        try:
            if not os.path.exists(path_param):
                # print(f"File model LSTM manual tidak ditemukan di: {path_param}")
                return False

            lstm_data = np.load(path_param, allow_pickle=True)
            self.Wf = lstm_data['Wf']; self.Wi = lstm_data['Wi']
            self.Wo = lstm_data['Wo']; self.Wc = lstm_data['Wc']
            self.bf = lstm_data['bf']; self.bi = lstm_data['bi']
            self.bo = lstm_data['bo']; self.bc = lstm_data['bc']
            self.Wy = lstm_data['Wy']; self.by = lstm_data['by']

            self.input_dim = int(lstm_data['input_dim'].item())
            self.hidden_dim = int(lstm_data['hidden_dim'].item())
            self.output_dim = int(lstm_data['output_dim'].item())
            self.learning_rate = float(lstm_data['learning_rate'].item())
            # print(f"Model LSTM manual berhasil dimuat dari: {path_param}")
            return True
        except Exception as e:
            print(f"Gagal memuat model LSTM manual dari {path_param}: {e}")
            return False

    def predict(self, x_input_vector):
        # x_input_vector adalah satu vektor (rata-rata embedding jawaban)
        # LSTM forward mengharapkan list of timesteps
        x_sequence_pred = [x_input_vector]
        y_pred_output = self.forward(x_sequence_pred)
        return y_pred_output # Ini akan jadi (output_dim, 1), misal (1,1)

print("Kelas LSTMManual dan fungsi pendukung telah didefinisikan.\n")

In [ ]:
# === Cell 2: Fungsi Persiapan Data untuk LSTM Manual ===
print("--- GRUP 3: SEL 2 ---")

def get_average_embedding_for_text(text_to_embed, embedding_model_param, default_vector_param):
    """
    Helper untuk mendapatkan vektor rata-rata dari teks menggunakan model embedding yang diberikan (bisa FastText atau KeyedVectors).
    """
    if embedding_model_param is None:
        return default_vector_param.copy()

    if not isinstance(text_to_embed, str) or not text_to_embed.strip():
        return default_vector_param.copy()

    words_processed = preprocess_text(text_to_embed) # Menggunakan preprocess_text global
    if not words_processed:
        return default_vector_param.copy()

    vectors_list = []
    # Cek apakah model adalah model FastText penuh atau KeyedVectors
    model_wv = None
    if hasattr(embedding_model_param, 'wv'): # Model FastText penuh
        model_wv = embedding_model_param.wv
    elif hasattr(embedding_model_param, 'key_to_index'): # Model KeyedVectors
        model_wv = embedding_model_param
    else: # Tidak dikenali
        return default_vector_param.copy()

    for word in words_processed:
        if word in model_wv:
            vectors_list.append(model_wv[word])

    if not vectors_list:
        return default_vector_param.copy()

    avg_vector = np.mean(vectors_list, axis=0)
    # Pastikan dimensi output benar
    if avg_vector.shape[0] != default_vector_param.shape[0]:
        # print(f"Peringatan: Dimensi avg_vector ({avg_vector.shape}) tidak cocok dengan default_vector ({default_vector_param.shape}). Menggunakan default.")
        return default_vector_param.copy()
    return avg_vector


def load_and_preprocess_data_for_lstm_manual(dataset_file_path,
                                              embedding_model_for_input, # Ini adalah ft_model_global
                                              mean_vector_for_input,   # Ini adalah mean_fasttext_vector_global
                                              test_split_size=0.2,
                                              random_state_split=42):
    """
    Memuat dataset, melakukan preprocessing, mengubah teks menjadi vektor rata-rata,
    dan membagi data untuk LSTM manual.
    Target Y akan dihitung sebagai cosine similarity antara jawaban dan soal+kunci (menggunakan embedding_model_large_vocab_global).
    """
    X_data_vectors = []
    Y_data_scores = []
    all_texts_for_lstm_input = [] # Untuk debugging atau analisis lebih lanjut

    if not os.path.exists(dataset_file_path):
        print(f"Error: File dataset '{dataset_file_path}' tidak ditemukan.")
        return np.array([]), np.array([]), np.array([]), np.array([])

    # Pastikan model embedding untuk input LSTM (ft_model_global) dan mean vector-nya valid
    if embedding_model_for_input is None or not hasattr(embedding_model_for_input, 'wv'):
        print("Error: Model embedding untuk input LSTM (ft_model) tidak valid.")
        return np.array([]), np.array([]), np.array([]), np.array([])
    if mean_vector_for_input is None or mean_vector_for_input.shape[0] != embedding_model_for_input.wv.vector_size:
        print(f"Error: Mean vector untuk input LSTM tidak valid atau dimensinya salah. Expected: {embedding_model_for_input.wv.vector_size}")
        return np.array([]), np.array([]), np.array([]), np.array([])

    # Model embedding besar untuk menghitung skor Y (cosine similarity soal+kunci vs jawaban)
    # embedding_model_large_vocab_global dan mean_vector_for_large_vocab_global harus tersedia global
    if embedding_model_large_vocab_global is None or mean_vector_for_large_vocab_global is None:
        print("Error: Model embedding besar (cc.id.300.vec) atau mean vector-nya tidak tersedia global untuk menghitung Y.")
        return np.array([]), np.array([]), np.array([]), np.array([])


    try:
        with open(dataset_file_path, 'r', encoding='utf-8') as f:
            dataset_list_g3 = json.load(f)
        print(f"Dataset berhasil dimuat dari '{dataset_file_path}' ({len(dataset_list_g3)} baris).")
    except Exception as e:
        print(f"Error saat memuat dataset: {e}")
        return np.array([]), np.array([]), np.array([]), np.array([])

    print("Memulai preprocessing teks dan pembuatan vektor untuk LSTM manual...")
    for i, item in enumerate(dataset_list_g3):
        soal = item.get('pertanyaan', "")
        jawaban_siswa = item.get('jawaban_siswa', "")
        kata_kunci_list_item = item.get('kata_kunci', [])

        if not (isinstance(soal, str) and soal.strip() and \
                isinstance(jawaban_siswa, str) and jawaban_siswa.strip()):
            # print(f"Data tidak lengkap pada item {i}, dilewati.")
            continue

        # Input X: Vektor rata-rata dari jawaban siswa (menggunakan ft_model_global)
        vektor_jawaban_siswa = get_average_embedding_for_text(jawaban_siswa,
                                                              embedding_model_for_input,
                                                              mean_vector_for_input)
        X_data_vectors.append(vektor_jawaban_siswa)

        # Target Y: Cosine similarity antara (vektor jawaban siswa) dan (vektor gabungan soal + kata kunci)
        # Untuk Y, kita gunakan embedding_model_large_vocab_global agar lebih kaya
        teks_referensi_y = soal
        if isinstance(kata_kunci_list_item, list) and kata_kunci_list_item:
            teks_referensi_y += " " + " ".join(kata_kunci_list_item)

        vektor_referensi_y = get_average_embedding_for_text(teks_referensi_y,
                                                            embedding_model_large_vocab_global,
                                                            mean_vector_for_large_vocab_global)
        # Vektor jawaban siswa untuk Y juga dihitung dengan model besar
        vektor_jawaban_siswa_y = get_average_embedding_for_text(jawaban_siswa,
                                                                embedding_model_large_vocab_global,
                                                                mean_vector_for_large_vocab_global)

        similarity_y = calculate_cosine_similarity(vektor_jawaban_siswa_y, vektor_referensi_y)
        Y_data_scores.append(similarity_y) # Skor similarity sebagai target

        # Simpan teks gabungan untuk X jika perlu (untuk referensi LSTM manual Anda)
        # Gabungkan pertanyaan, jawaban, kunci menjadi satu teks input per sampel untuk X
        # (Pendekatan dari notebook asli Anda untuk input LSTM Keras)
        # Untuk LSTM manual yang menerima satu vektor, kita sudah buat vektor_jawaban_siswa di atas
        # Jika Anda ingin X adalah gabungan:
        # kata_kunci_str = " ".join(kata_kunci_list_item) if isinstance(kata_kunci_list_item, list) else str(kata_kunci_list_item)
        # combined_text_for_X = f"PERTANYAAN: {' '.join(preprocess_text(soal))} JAWABAN: {' '.join(preprocess_text(jawaban_siswa))} KUNCI: {' '.join(preprocess_text(kata_kunci_str))}"
        # X_vector = get_average_embedding_for_text(combined_text_for_X, embedding_model_for_input, mean_vector_for_input)
        # X_data_vectors.append(X_vector)

    if not X_data_vectors or not Y_data_scores:
        print("Tidak ada data valid yang diproses untuk X atau Y.")
        return np.array([]), np.array([]), np.array([]), np.array([])

    X_data_np = np.array(X_data_vectors, dtype=np.float32)
    Y_data_np = np.array(Y_data_scores, dtype=np.float32).reshape(-1, 1) # Pastikan 2D untuk Y

    print(f"Preprocessing dan pembuatan vektor selesai. Shape X: {X_data_np.shape}, Shape Y: {Y_data_np.shape}")

    # Pembagian data (menggunakan fungsi train_test_split dari sklearn sesuai daftar Anda)
    X_train, X_test, y_train, y_test = train_test_split(
        X_data_np, Y_data_np, test_size=test_split_size, random_state=random_state_split, stratify=None
    )
    print(f"Data berhasil di-split: Train X:{X_train.shape}, Y:{y_train.shape}; Test X:{X_test.shape}, Y:{y_test.shape}")

    return X_train, X_test, y_train, y_test

print("Fungsi persiapan data untuk LSTM manual telah didefinisikan.\n")

In [ ]:
# === Cell 3: Memanggil Fungsi Persiapan Data ===
print("--- GRUP 3: SEL 3 ---")

# Pastikan ft_model_global dan mean_fasttext_vector_global sudah siap dari Grup 2
# dataset_path_global dari Grup 1

X_train_lstm_manual, X_test_lstm_manual, y_train_lstm_manual, y_test_lstm_manual = np.array([]), np.array([]), np.array([]), np.array([])

if ft_model_global is not None and hasattr(ft_model_global, 'wv') and \
   mean_fasttext_vector_global is not None and mean_fasttext_vector_global.shape[0] > 0 and \
   embedding_model_large_vocab_global is not None and mean_vector_for_large_vocab_global is not None :

    print("Memanggil load_and_preprocess_data_for_lstm_manual...")
    X_train_lstm_manual, X_test_lstm_manual, \
    y_train_lstm_manual, y_test_lstm_manual = load_and_preprocess_data_for_lstm_manual(
        dataset_file_path=dataset_path_global,
        embedding_model_for_input=ft_model_global, # Model fine-tuned untuk input X
        mean_vector_for_input=mean_fasttext_vector_global,
        test_split_size=0.2,
        random_state_split=42
    )

    if X_train_lstm_manual.size > 0:
        print("\nContoh data setelah persiapan:")
        print(f"  X_train_lstm_manual[0][:5]: {X_train_lstm_manual[0][:5]}")
        print(f"  y_train_lstm_manual[0]: {y_train_lstm_manual[0]}")
    else:
        print("Tidak ada data training yang dihasilkan.")
else:
    print("PERINGATAN: Model embedding (ft_model_global atau embedding_model_large_vocab_global) atau mean vector tidak tersedia. Persiapan data LSTM manual tidak dapat dilanjutkan.")

print("Pemanggilan fungsi persiapan data selesai.\n")

In [ ]:
# === Cell 4: Inisialisasi atau Pemuatan Model LSTM Manual ===
print("--- GRUP 3: SEL 4 ---")

# lstm_manual_model_instance_global diinisialisasi di Grup 1
# lstm_manual_model_file_path_global dari Grup 1
# ft_model_global dari Grup 2

# Tentukan dimensi LSTM
LSTM_INPUT_DIM_G3 = 300 # Default, akan diperbarui dari embedding model
if ft_model_global is not None and hasattr(ft_model_global, 'wv'):
    LSTM_INPUT_DIM_G3 = ft_model_global.wv.vector_size
elif embedding_model_large_vocab_global is not None: # Fallback ke model besar jika ft_model tidak ada
    LSTM_INPUT_DIM_G3 = embedding_model_large_vocab_global.vector_size

LSTM_HIDDEN_DIM_G3 = 128  # Jumlah hidden unit (bisa disesuaikan)
LSTM_OUTPUT_DIM_G3 = 1    # Output tunggal (skor)
LSTM_LEARNING_RATE_G3 = 0.005 # Learning rate (bisa disesuaikan)
LSTM_EPOCHS_G3 = 20       # Jumlah epoch training (bisa disesuaikan)
LSTM_BATCH_SIZE_G3 = 16   # Batch size untuk training

lstm_manual_model_instance_global = LSTMManual(
    input_dim=LSTM_INPUT_DIM_G3,
    hidden_dim=LSTM_HIDDEN_DIM_G3,
    output_dim=LSTM_OUTPUT_DIM_G3,
    learning_rate=LSTM_LEARNING_RATE_G3
)
print(f"Instance LSTMManual dibuat dengan input_dim={LSTM_INPUT_DIM_G3}, hidden_dim={LSTM_HIDDEN_DIM_G3}.")

lstm_model_loaded_successfully_g3 = False
if lstm_manual_model_instance_global.load_model(lstm_manual_model_file_path_global):
    # Cek apakah dimensi model yang dimuat sesuai
    metadata_lstm_loaded = load_lstm_manual_metadata(lstm_manual_metadata_file_path_global)
    if metadata_lstm_loaded and \
       metadata_lstm_loaded.get('input_dim') == LSTM_INPUT_DIM_G3 and \
       metadata_lstm_loaded.get('hidden_dim') == LSTM_HIDDEN_DIM_G3 and \
       metadata_lstm_loaded.get('output_dim') == LSTM_OUTPUT_DIM_G3:
        print(f"Model LSTM manual ditemukan dan berhasil dimuat dari: {lstm_manual_model_file_path_global}")
        print("  Parameter model yang dimuat cocok dengan konfigurasi saat ini.")
        lstm_model_loaded_successfully_g3 = True
    else:
        if metadata_lstm_loaded:
            print(f"Model LSTM manual dimuat, TAPI parameter tidak cocok. Konfigurasi saat ini: In={LSTM_INPUT_DIM_G3}, Hid={LSTM_HIDDEN_DIM_G3}. Model: In={metadata_lstm_loaded.get('input_dim')}, Hid={metadata_lstm_loaded.get('hidden_dim')}.")
        else:
            print("Model LSTM manual dimuat, tapi metadata tidak ditemukan atau tidak valid.")
        print("  Akan melatih ulang model LSTM.")
        # Reset bobot jika parameter tidak cocok
        lstm_manual_model_instance_global = LSTMManual(
            input_dim=LSTM_INPUT_DIM_G3, hidden_dim=LSTM_HIDDEN_DIM_G3, output_dim=LSTM_OUTPUT_DIM_G3, learning_rate=LSTM_LEARNING_RATE_G3
        )
else:
    print(f"File model LSTM manual tidak ditemukan di {lstm_manual_model_file_path_global} atau gagal dimuat.")

print(f"Status pemuatan model LSTM manual berhasil: {lstm_model_loaded_successfully_g3}\n")

In [ ]:
# === Cell 5: Training dan Penyimpanan Model LSTM Manual ===
print("--- GRUP 3: SEL 5 ---")

if not lstm_model_loaded_successfully_g3:
    if X_train_lstm_manual.size > 0 and y_train_lstm_manual.size > 0:
        print(f"Memulai training LSTM manual dengan {LSTM_EPOCHS_G3} epoch, batch size {LSTM_BATCH_SIZE_G3}...")
        print(f"  Input X_train shape: {X_train_lstm_manual.shape}, Y_train shape: {y_train_lstm_manual.shape}")

        lstm_manual_model_instance_global.train(
            X_train_lstm_manual,
            y_train_lstm_manual,
            epochs=LSTM_EPOCHS_G3,
            batch_size=LSTM_BATCH_SIZE_G3
        )
        print("Training LSTM manual selesai.")

        lstm_manual_model_instance_global.save_model(lstm_manual_model_file_path_global)

        metadata_lstm_to_save = {
            'train_size_samples': X_train_lstm_manual.shape[0],
            'input_dim': lstm_manual_model_instance_global.input_dim,
            'hidden_dim': lstm_manual_model_instance_global.hidden_dim,
            'output_dim': lstm_manual_model_instance_global.output_dim,
            'learning_rate': lstm_manual_model_instance_global.learning_rate,
            'epochs_trained': LSTM_EPOCHS_G3,
            'batch_size_trained': LSTM_BATCH_SIZE_G3,
            'model_type': 'LSTM_Manual_NumPy',
            'training_date': datetime.now().isoformat()
        }
        save_lstm_manual_metadata(lstm_manual_metadata_file_path_global, metadata_lstm_to_save)
        print("Model LSTM manual dan metadata baru telah disimpan.")
    else:
        print("PERINGATAN: Data training (X_train_lstm_manual atau y_train_lstm_manual) kosong. Training LSTM manual dilewati.")
else:
    print("Training LSTM manual dilewati karena model sudah dimuat.")

print("Proses training/penyimpanan model LSTM manual selesai.\n")

In [ ]:
# === Cell 6: Fungsi Evaluasi LSTM Manual ===
print("--- GRUP 3: SEL 6 ---")

def evaluate_lstm_manual(model_lstm, X_test_data, y_test_data):
    """Mengevaluasi model LSTM manual pada data test."""
    if model_lstm is None:
        print("Model LSTM manual tidak tersedia untuk evaluasi.")
        return None

    if X_test_data.size == 0 or y_test_data.size == 0:
        print("Data test (X_test atau y_test) kosong. Evaluasi LSTM manual tidak dapat dilakukan.")
        return None

    y_preds_list = []
    for i in range(X_test_data.shape[0]):
        x_sample_eval = X_test_data[i]
        # LSTM forward mengharapkan list of timesteps
        y_pred_eval = model_lstm.predict(x_sample_eval) # predict sudah menghandle [x_sample_eval]
        y_preds_list.append(y_pred_eval.item()) # Ambil nilai skalar

    y_preds_np = np.array(y_preds_list)
    y_true_np = y_test_data.flatten() # Pastikan y_true juga 1D

    if y_preds_np.shape != y_true_np.shape:
        print(f"PERINGATAN Evaluasi: Shape y_preds ({y_preds_np.shape}) dan y_true ({y_true_np.shape}) tidak cocok.")
        return None


    mse = mean_squared_error(y_true_np, y_preds_np)
    mae = mean_absolute_error(y_true_np, y_preds_np)
    r2 = r2_score(y_true_np, y_preds_np)
    rmse = sqrt(mse)

    # Pearson dan Spearman
    pearson_corr, _ = pearsonr(y_true_np, y_preds_np) if len(y_true_np) > 1 else (0,0)
    spearman_corr, _ = spearmanr(y_true_np, y_preds_np) if len(y_true_np) > 1 else (0,0)


    print("\n--- Hasil Evaluasi Model LSTM Manual pada Data Test ---")
    print(f"  Mean Squared Error (MSE)    : {mse:.4f}")
    print(f"  Root Mean Squared Error (RMSE): {rmse:.4f}")
    print(f"  Mean Absolute Error (MAE)   : {mae:.4f}")
    print(f"  R^2 Score                   : {r2:.4f}")
    print(f"  Pearson Correlation         : {pearson_corr:.4f}")
    print(f"  Spearman Correlation        : {spearman_corr:.4f}")


    # Contoh Prediksi
    num_samples_to_show_eval = min(5, X_test_data.shape[0])
    if num_samples_to_show_eval > 0:
        print("\n  Contoh Prediksi vs Aktual:")
        print(f"  {'Index':<7} | {'Aktual':<10} | {'Prediksi':<10}")
        print("  " + "-" * 30)
        sample_indices_eval = np.random.choice(X_test_data.shape[0], num_samples_to_show_eval, replace=False)
        for idx_sample in sample_indices_eval:
            actual_val = y_true_np[idx_sample]
            predicted_val = y_preds_np[idx_sample]
            print(f"  {idx_sample:<7} | {actual_val:<10.4f} | {predicted_val:<10.4f}")

    evaluation_metrics_dict = {
        'mse': mse, 'rmse': rmse, 'mae': mae, 'r2': r2,
        'pearson': pearson_corr, 'spearman': spearman_corr
    }
    return evaluation_metrics_dict

# Panggil evaluasi jika model dan data test ada
if lstm_manual_model_instance_global is not None and \
   X_test_lstm_manual.size > 0 and y_test_lstm_manual.size > 0:
    print("\nMengevaluasi model LSTM manual pada data test...")
    lstm_eval_results = evaluate_lstm_manual(lstm_manual_model_instance_global, X_test_lstm_manual, y_test_lstm_manual)
    if lstm_eval_results:
        # Simpan hasil evaluasi ke log jika perlu
        try:
            # Path dari Grup 1: lstm_training_log_file_global
            # Kita bisa tambahkan hasil evaluasi ini ke log yang ada atau buat file baru
            log_file_eval_path = os.path.join(LOGS_DIR, 'lstm_manual_evaluation_log.json')
            with open(log_file_eval_path, 'w') as f_log_eval:
                json.dump({
                    "evaluation_time": datetime.now().isoformat(),
                    "metrics": lstm_eval_results,
                    "model_file": lstm_manual_model_file_path_global
                }, f_log_eval, indent=4)
            print(f"Hasil evaluasi LSTM manual disimpan di: {log_file_eval_path}")
        except Exception as e:
            print(f"Gagal menyimpan log evaluasi LSTM manual: {e}")
else:
    print("Model LSTM manual atau data test tidak tersedia untuk evaluasi.")

print("Fungsi evaluasi LSTM manual selesai.\n")

# **IMPLEMENTASI ALGORITMA PENILAIAN**

---



In [ ]:
# --- GRUP 4: IMPLEMENTASI ALGORITMA PENILAIAN ---

# === Cell 1: Pengecekan Variabel Global yang Dibutuhkan ===
print("--- GRUP 4: SEL 1 ---")

# Variabel-variabel ini seharusnya sudah ada dari Grup 1, 2, dan 3:
# - stemmer_global, active_stopwords_list_global (dari Grup 1, Sel 6)
# - ft_model_global (dari Grup 2, Sel 9 atau Sel 5 jika dimuat)
# - mean_fasttext_vector_global (dari Grup 2, Sel 9 atau Sel 5 jika dimuat)
# - embedding_model_large_vocab_global (dari Grup 2, Sel 4)
# - mean_vector_for_large_vocab_global (dari Grup 2, Sel 4)
# - fungsi preprocess_text (dari Grup 2, Sel 1)
# - fungsi calculate_cosine_similarity (dari Grup 2, Sel 1)

# Pengecekan sederhana
if 'preprocess_text' not in globals():
    print("PERINGATAN: Fungsi 'preprocess_text' tidak terdefinisi secara global.")
if 'calculate_cosine_similarity' not in globals():
    print("PERINGATAN: Fungsi 'calculate_cosine_similarity' tidak terdefinisi secara global.")
if 'ft_model_global' not in globals() or ft_model_global is None:
    print("PERINGATAN: 'ft_model_global' (model embedding fine-tuned) tidak tersedia.")
if 'embedding_model_large_vocab_global' not in globals() or embedding_model_large_vocab_global is None:
    print("PERINGATAN: 'embedding_model_large_vocab_global' (cc.id.300.vec) tidak tersedia.")

print("Pengecekan variabel global untuk Grup 4 selesai.\n")

In [ ]:
# === Cell 2: Implementasi ROUGE Score ===
print("--- GRUP 4: SEL 2 ---")

def rouge_score_g4(reference_text_param, student_text_param):
    """
    Menghitung ROUGE-1 F1-score antara teks referensi dan teks siswa.
    Menggunakan preprocess_text global.
    """
    # Pastikan input adalah string dan lakukan preprocessing
    ref_words_g4 = preprocess_text(str(reference_text_param))
    stu_words_g4 = preprocess_text(str(student_text_param))

    if not ref_words_g4 or not stu_words_g4:
        return 0.0

    common_unigrams_g4 = set(ref_words_g4) & set(stu_words_g4)

    # Hindari ZeroDivisionError jika salah satu list kata kosong (meskipun sudah dicek di atas)
    len_ref_words = len(ref_words_g4)
    len_stu_words = len(stu_words_g4)

    if len_ref_words == 0 or len_stu_words == 0: # Seharusnya tidak terjadi jika cek awal lolos
        return 0.0

    recall_g4 = len(common_unigrams_g4) / len_ref_words
    precision_g4 = len(common_unigrams_g4) / len_stu_words

    if (recall_g4 + precision_g4) == 0:
        return 0.0
    f1_score_rouge_g4 = 2 * (recall_g4 * precision_g4) / (recall_g4 + precision_g4)

    return f1_score_rouge_g4

print("Fungsi rouge_score_g4 telah didefinisikan.\n")

In [ ]:
# === Cell 3: Implementasi TF-IDF Similarity ===
print("--- GRUP 4: SEL 3 ---")

def tfidf_similarity_g4(reference_text_param, student_text_param, context_docs_param=None):
    """
    Menghitung kemiripan kosinus TF-IDF.
    Menggunakan preprocess_text global dan calculate_cosine_similarity global.
    """
    ref_words_g4 = preprocess_text(str(reference_text_param))
    stu_words_g4 = preprocess_text(str(student_text_param))

    if not ref_words_g4 or not stu_words_g4:
        return 0.0

    vocabulary_g4 = set(ref_words_g4) | set(stu_words_g4)
    if not vocabulary_g4: # Jika vocabulary kosong (misal semua kata adalah stopwords)
        return 0.0


    tf_ref_g4 = defaultdict(int)
    for word in ref_words_g4: tf_ref_g4[word] += 1

    tf_stu_g4 = defaultdict(int)
    for word in stu_words_g4: tf_stu_g4[word] += 1

    idf_g4 = defaultdict(float)
    num_docs_in_context_g4 = 0
    processed_context_docs_g4 = []

    if context_docs_param and isinstance(context_docs_param, list):
        num_docs_in_context_g4 = len(context_docs_param)
        processed_context_docs_g4 = [set(preprocess_text(str(doc))) for doc in context_docs_param]

    if num_docs_in_context_g4 == 0:
        # print("Peringatan TF-IDF: Tidak ada 'context_docs_param' yang valid. IDF akan default.")
        for word in vocabulary_g4: idf_g4[word] = 1.0 # Default IDF
    else:
        for word in vocabulary_g4:
            docs_containing_word_g4 = sum(1 for processed_doc_set in processed_context_docs_g4 if word in processed_doc_set)
            idf_g4[word] = math.log((num_docs_in_context_g4 + 1) / (docs_containing_word_g4 + 1.0)) + 1.0 # Smoothing, tambah 1 agar non-negatif

    sorted_vocabulary_g4 = sorted(list(vocabulary_g4))

    # Normalisasi TF sebelum dikalikan dengan IDF
    len_ref = len(ref_words_g4)
    len_stu = len(stu_words_g4)

    tfidf_vector_ref_g4 = np.array([(tf_ref_g4[word] / len_ref if len_ref > 0 else 0) * idf_g4[word] for word in sorted_vocabulary_g4])
    tfidf_vector_stu_g4 = np.array([(tf_stu_g4[word] / len_stu if len_stu > 0 else 0) * idf_g4[word] for word in sorted_vocabulary_g4])

    return calculate_cosine_similarity(tfidf_vector_ref_g4, tfidf_vector_stu_g4) # Menggunakan fungsi dari Grup 2

print("Fungsi tfidf_similarity_g4 telah didefinisikan.\n")

In [ ]:
# === Cell 4: Fungsi Konversi Teks ke Vektor Embedding (untuk Cosine Similarity) ===
# Menggunakan kembali fungsi text_to_vector_for_similarity dari Grup 2, Sel 1
# atau definisikan ulang di sini jika ada penyesuaian spesifik untuk Grup 4.
# Untuk konsistensi, lebih baik menggunakan yang sudah ada jika fungsinya sama.
# Kita asumsikan text_to_vector_for_similarity dari Grup 2, Sel 1 sudah cukup.
print("--- GRUP 4: SEL 4 ---")

# Fungsi text_to_vector_for_cosine_eval akan menjadi alias atau pemanggilan
# ke text_to_vector_for_similarity yang menggunakan model embedding besar (cc.id.300.vec)
# Fungsi ini akan digunakan dalam evaluate_answer di Grup 5.

def text_to_vector_for_g4_eval(text_input_param,
                               embedding_model_param, # Akan diisi dengan embedding_model_large_vocab_global
                               default_vector_param): # Akan diisi dengan mean_vector_for_large_vocab_global
    """
    Wrapper/alias untuk text_to_vector_for_similarity.
    Tujuan utamanya adalah untuk memastikan model dan default vector yang benar digunakan
    saat menghitung cosine similarity di fungsi penilaian akhir.
    """
    return text_to_vector_for_similarity(text_input_param, embedding_model_param, default_vector_param)

print("Fungsi text_to_vector_for_g4_eval (menggunakan text_to_vector_for_similarity) siap.\n")

# Fungsi untuk mendapatkan vektor input LSTM (menggunakan ft_model_global)
# Ini mirip dengan get_average_embedding_for_text dari Grup 3, Sel 2
def get_lstm_input_vector_g4(text_input_param):
    """
    Menghasilkan vektor rata-rata dari teks untuk input LSTM manual.
    Menggunakan ft_model_global dan mean_fasttext_vector_global.
    """
    # Pastikan model dan mean vector global tersedia
    if 'ft_model_global' not in globals() or ft_model_global is None or \
       'mean_fasttext_vector_global' not in globals() or mean_fasttext_vector_global is None:
        print("PERINGATAN (get_lstm_input_vector_g4): ft_model_global atau mean_fasttext_vector_global tidak tersedia.")
        # Tentukan dimensi fallback
        fallback_dim = 300
        if 'ft_model_global' in globals() and ft_model_global is not None and hasattr(ft_model_global, 'wv'):
            fallback_dim = ft_model_global.wv.vector_size
        elif 'embedding_model_large_vocab_global' in globals() and embedding_model_large_vocab_global is not None:
             fallback_dim = embedding_model_large_vocab_global.vector_size
        return np.zeros(fallback_dim)


    return get_average_embedding_for_text(text_input_param,
                                          ft_model_global, # Model fine-tuned untuk input LSTM
                                          mean_fasttext_vector_global)


print("Fungsi get_lstm_input_vector_g4 (untuk input LSTM manual) telah didefinisikan.\n")

# **INPUT DATA DAN EVALUASI**

---



In [ ]:
# --- GRUP 5: INPUT DATA DAN EVALUASI ---

# === Cell 1: Persiapan Komponen Evaluasi Akhir ===
print("--- GRUP 5: SEL 1 ---")

# Variabel-variabel ini seharusnya sudah siap dari grup-grup sebelumnya:
# - embedding_model_large_vocab_global (cc.id.300.vec dari Grup 2, Sel 4)
# - mean_vector_for_large_vocab_global (dari Grup 2, Sel 4)
# - ft_model_global (model "glove_finetuned" dari Grup 2, Sel 9 atau Sel 5)
# - mean_fasttext_vector_global (dari Grup 2, Sel 9 atau Sel 5)
# - lstm_manual_model_instance_global (dari Grup 3, Sel 4 atau Sel 5)
# - Fungsi-fungsi dari Grup 2 dan Grup 4 (preprocess_text, calculate_cosine_similarity,
#   text_to_vector_for_g4_eval, get_lstm_input_vector_g4, rouge_score_g4, tfidf_similarity_g4)

# Pengecekan komponen krusial untuk evaluasi
komponen_evaluasi_siap = True
if 'embedding_model_large_vocab_global' not in globals() or embedding_model_large_vocab_global is None:
    print("PERINGATAN: 'embedding_model_large_vocab_global' tidak tersedia.")
    komponen_evaluasi_siap = False
if 'mean_vector_for_large_vocab_global' not in globals() or mean_vector_for_large_vocab_global is None:
    print("PERINGATAN: 'mean_vector_for_large_vocab_global' tidak tersedia.")
    komponen_evaluasi_siap = False
if 'ft_model_global' not in globals() or ft_model_global is None:
    print("PERINGATAN: 'ft_model_global' (untuk input LSTM & beberapa similarity) tidak tersedia.")
    komponen_evaluasi_siap = False
if 'mean_fasttext_vector_global' not in globals() or mean_fasttext_vector_global is None:
    print("PERINGATAN: 'mean_fasttext_vector_global' tidak tersedia.")
    komponen_evaluasi_siap = False
if 'lstm_manual_model_instance_global' not in globals() or lstm_manual_model_instance_global is None:
    print("PERINGATAN: 'lstm_manual_model_instance_global' tidak tersedia.")
    komponen_evaluasi_siap = False

if komponen_evaluasi_siap:
    print("Semua komponen model dan fungsi pendukung untuk evaluasi akhir tampaknya siap.")
else:
    print("SATU ATAU LEBIH KOMPONEN KRUSIAL TIDAK SIAP. EVALUASI MUNGKIN GAGAL ATAU TIDAK AKURAT.")

print("Persiapan komponen evaluasi akhir selesai.\n")

In [ ]:
# === Cell 2: Definisi Fungsi evaluate_answer (dengan LSTM Manual dan Dua Threshold Scaling) ===
print("--- GRUP 5: SEL 2 ---")

def evaluate_answer_g5(jawaban_siswa_text_param, soal_text_param, kata_kunci_list_param,
                       embedding_model_for_cosine_param,   # Akan diisi dengan embedding_model_large_vocab_global
                       mean_vector_for_cosine_param,     # Akan diisi dengan mean_vector_for_large_vocab_global
                       embedding_model_for_lstm_input,   # Akan diisi dengan ft_model_global
                       mean_vector_for_lstm_input,       # Akan diisi dengan mean_fasttext_vector_global
                       lstm_model_param,                 # Akan diisi dengan lstm_manual_model_instance_global
                       konteks_tfidf_dokumen=None # Opsional: list string dokumen untuk IDF yang lebih baik
                       ):
    """
    Mengevaluasi jawaban siswa dengan bobot dinamis dan dua kemungkinan threshold scaling,
    menggunakan LSTM manual dan model embedding yang ditentukan.
    """
    if not all([embedding_model_for_cosine_param, lstm_model_param,
                mean_vector_for_cosine_param is not None,
                embedding_model_for_lstm_input is not None, # Cek model untuk input LSTM
                mean_vector_for_lstm_input is not None]): # Cek mean vector untuk input LSTM
        print("  Peringatan di evaluate_answer_g5: Model/mean_vector penting tidak valid. Mengembalikan skor 0.")
        return 0.0

    # Teks referensi untuk ROUGE dan TF-IDF adalah gabungan soal dan kata kunci
    # Namun, untuk cosine similarity, kita akan bandingkan jawaban dengan soal saja (atau bisa disesuaikan)
    # Sesuai notebook awal, teks referensi untuk ROUGE & TF-IDF adalah soal.
    reference_text_for_metrics_g5 = str(soal_text_param)
    jawaban_siswa_str = str(jawaban_siswa_text_param)

    # 1. Hitung Skor ROUGE (menggunakan fungsi dari Grup 4)
    skor_rouge_val = rouge_score_g4(reference_text_for_metrics_g5, jawaban_siswa_str)

    # 2. Hitung Skor TF-IDF (menggunakan fungsi dari Grup 4)
    # Konteks untuk TF-IDF bisa dari soal dan kata kunci
    if konteks_tfidf_dokumen is None: # Default konteks jika tidak diberikan
        konteks_tfidf_dokumen_internal = [reference_text_for_metrics_g5]
        if isinstance(kata_kunci_list_param, list):
            konteks_tfidf_dokumen_internal.extend([str(kw) for kw in kata_kunci_list_param if isinstance(kw, str) and kw.strip()])
    else:
        konteks_tfidf_dokumen_internal = konteks_tfidf_dokumen # Gunakan konteks yang diberikan jika ada

    skor_tfidf_val = tfidf_similarity_g4(reference_text_for_metrics_g5, jawaban_siswa_str,
                                         context_docs_param=konteks_tfidf_dokumen_internal)

    # 3. Hitung Skor Cosine Embedding (menggunakan embedding_model_large_vocab_global)
    # Vektor jawaban dan vektor soal (sebagai referensi) dihitung dengan model embedding besar
    vector_jawaban_cosine = text_to_vector_for_g4_eval(jawaban_siswa_str,
                                                       embedding_model_for_cosine_param,
                                                       mean_vector_for_cosine_param)
    vector_ref_cosine = text_to_vector_for_g4_eval(reference_text_for_metrics_g5, # Referensi cosine adalah soal
                                                   embedding_model_for_cosine_param,
                                                   mean_vector_for_cosine_param)
    skor_cosine_embedding_val = calculate_cosine_similarity(vector_jawaban_cosine, vector_ref_cosine)


    # 4. Dapatkan Skor dari Model LSTM Manual
    # Input untuk LSTM manual adalah vektor rata-rata dari jawaban siswa (menggunakan ft_model_global)
    vektor_jawaban_for_lstm = get_lstm_input_vector_g4(jawaban_siswa_str) # Fungsi dari Grup 4, Sel 4
    skor_lstm_pred_val = 0.0
    if np.all(vektor_jawaban_for_lstm == 0): # Jika vektor jawaban nol (misal, semua kata OOV atau jawaban kosong)
        # print("  Peringatan LSTM: Vektor input untuk LSTM adalah nol. Skor LSTM diatur ke 0.")
        pass # skor_lstm_pred_val sudah 0.0
    else:
        try:
            # LSTMManual.predict() mengharapkan satu vektor, bukan list of sequences
            pred_output_lstm = lstm_model_param.predict(vektor_jawaban_for_lstm)
            skor_lstm_pred_val = pred_output_lstm.item() # Ambil nilai skalar dari output (1,1)
        except Exception as e:
            print(f"  Error saat prediksi LSTM manual: {e}. Skor LSTM diatur ke 0.")
            skor_lstm_pred_val = 0.0

    # --- Logika Pembobotan Dinamis (Sesuai notebook asli Anda) ---
    if skor_rouge_val <= 0.01 and skor_tfidf_val <= 0.01:
        bobot_rouge_final_g5 = 0.00
        bobot_tfidf_final_g5 = 0.00
        bobot_cosine_final_g5 = 0.40
        bobot_lstm_final_g5 = 0.60
    else:
        bobot_rouge_final_g5 = 0.05
        bobot_tfidf_final_g5 = 0.05
        bobot_cosine_final_g5 = 0.25
        bobot_lstm_final_g5 = 0.65

    skor_akhir_gabungan_g5 = (skor_rouge_val * bobot_rouge_final_g5) + \
                             (skor_tfidf_val * bobot_tfidf_final_g5) + \
                             (skor_cosine_embedding_val * bobot_cosine_final_g5) + \
                             (skor_lstm_pred_val * bobot_lstm_final_g5)
    skor_akhir_gabungan_g5 = max(0.0, min(1.0, skor_akhir_gabungan_g5))
    skor_sebelum_scaling_g5 = skor_akhir_gabungan_g5

    # --- STRATEGI SCALING NON-LINEAR DENGAN DUA KEMUNGKINAN THRESHOLD ---
    skor_setelah_scaling_g5 = skor_akhir_gabungan_g5
    threshold_scaling_menengah_g5 = 0.55
    faktor_angkat_menengah_g5 = 0.20
    threshold_scaling_tinggi_g5 = 0.62
    faktor_angkat_tinggi_g5 = 0.40

    scaling_info = ""
    if skor_akhir_gabungan_g5 > threshold_scaling_tinggi_g5:
        sisa_jarak_ke_satu_g5 = 1.0 - skor_akhir_gabungan_g5
        penambahan_skor_g5 = sisa_jarak_ke_satu_g5 * faktor_angkat_tinggi_g5
        skor_setelah_scaling_g5 = skor_akhir_gabungan_g5 + penambahan_skor_g5
        scaling_info = f"Info Scaling Tinggi: Skor Gab. Awal={skor_sebelum_scaling_g5:.4f} > Thresh={threshold_scaling_tinggi_g5:.2f}. Dinaikkan (Faktor: {faktor_angkat_tinggi_g5:.2f})"
    elif skor_akhir_gabungan_g5 > threshold_scaling_menengah_g5:
        sisa_jarak_ke_satu_g5 = 1.0 - skor_akhir_gabungan_g5
        penambahan_skor_g5 = sisa_jarak_ke_satu_g5 * faktor_angkat_menengah_g5
        skor_setelah_scaling_g5 = skor_akhir_gabungan_g5 + penambahan_skor_g5
        scaling_info = f"Info Scaling Menengah: Skor Gab. Awal={skor_sebelum_scaling_g5:.4f} > Thresh={threshold_scaling_menengah_g5:.2f}. Dinaikkan (Faktor: {faktor_angkat_menengah_g5:.2f})"

    skor_akhir_final_g5 = max(0.0, min(1.0, skor_setelah_scaling_g5))

    # Cetak detail skor dan info scaling jika ada
    if scaling_info:
        print(f"    {scaling_info}")
    print(f"    Detail Skor: ROUGE={skor_rouge_val:.4f} (B:{bobot_rouge_final_g5:.2f}), "
          f"TF-IDF={skor_tfidf_val:.4f} (B:{bobot_tfidf_final_g5:.2f}), "
          f"CosEmb={skor_cosine_embedding_val:.4f} (B:{bobot_cosine_final_g5:.2f}), "
          f"LSTM={skor_lstm_pred_val:.4f} (B:{bobot_lstm_final_g5:.2f}) -> Gabungan={skor_sebelum_scaling_g5:.4f} -> Scaled Akhir={skor_akhir_final_g5:.4f}")

    return float(skor_akhir_final_g5)

print("Fungsi evaluate_answer_g5 (dengan LSTM Manual dan Dua Threshold Scaling) telah didefinisikan.\n")

In [ ]:
# === Cell 3: Membaca dan Mem-parsing File input_soal.txt ===
print("--- GRUP 5: SEL 3 ---")

def parse_input_soal_file_g5(file_path_param):
    """
    Membaca dan mem-parsing file input_soal.txt untuk mendapatkan daftar tugas evaluasi.
    """
    tasks_parsed = []
    # Pastikan file_path_param adalah string dan file ada
    if not isinstance(file_path_param, str) or not os.path.exists(file_path_param):
        print(f"  Error: Path file input '{file_path_param}' tidak valid atau file tidak ditemukan.")
        return tasks_parsed

    try:
        with open(file_path_param, 'r', encoding='utf-8') as f_input:
            content_input = f_input.read()
    except Exception as e:
        print(f"  Error saat membaca file {file_path_param}: {e}")
        return tasks_parsed

    # Membersihkan karakter '\' yang mungkin diikuti spasi (sering muncul dari copy-paste)
    content_cleaned_input = re.sub(r'\\\s*', '', content_input)
    # Memisahkan blok soal berdasarkan dua atau lebih baris baru
    blocks_input = re.split(r'\n\s*\n+', content_cleaned_input.strip())

    for i_block, block_text in enumerate(blocks_input):
        if not block_text.strip(): continue

        current_task_item = {
            "id_soal": f"InputFile_{i_block+1:03d}",
            "pertanyaan": "",
            "jawaban_siswa": "",
            "kata_kunci": []
        }
        for line_text in block_text.strip().split('\n'):
            line_text_stripped = line_text.strip()
            if line_text_stripped.lower().startswith("pertanyaan:"):
                current_task_item["pertanyaan"] = line_text_stripped.split(":", 1)[1].strip()
            elif line_text_stripped.lower().startswith("jawaban:"):
                current_task_item["jawaban_siswa"] = line_text_stripped.split(":", 1)[1].strip()
            elif line_text_stripped.lower().startswith("kata kunci:"):
                keywords_str_input = line_text_stripped.split(":", 1)[1].strip()
                current_task_item["kata_kunci"] = [kw.strip() for kw in keywords_str_input.split(',') if kw.strip()]

        if current_task_item["pertanyaan"] and current_task_item["jawaban_siswa"]:
            tasks_parsed.append(current_task_item)
        else:
            print(f"  Peringatan: Blok soal ke-{i_block+1} di file '{os.path.basename(file_path_param)}' tidak memiliki 'Pertanyaan:' atau 'Jawaban:' yang valid. Dilewati.")

    return tasks_parsed

# Path dari Grup 1: input_soal_txt_path_global
print(f"Membaca dan mem-parsing file input: {input_soal_txt_path_global}")
tasks_to_evaluate_final_g5 = parse_input_soal_file_g5(input_soal_txt_path_global)

if not tasks_to_evaluate_final_g5:
    print("Tidak ada tugas evaluasi yang berhasil diparsing. Periksa file input atau path.")
else:
    print(f"Berhasil mem-parsing {len(tasks_to_evaluate_final_g5)} tugas evaluasi dari '{os.path.basename(input_soal_txt_path_global)}'.")
    if tasks_to_evaluate_final_g5:
        print("Contoh tugas pertama yang diparsing:")
        # Cetak dengan json.dumps untuk format yang lebih rapi, pastikan ensure_ascii=False untuk karakter non-ASCII
        print(json.dumps(tasks_to_evaluate_final_g5[0], indent=2, ensure_ascii=False))
print("Parsing file input selesai.\n")

In [ ]:
# === Cell 4: Loop Evaluasi dari Data yang Diparsing dan Penyimpanan Hasil ===
print("--- GRUP 5: SEL 4 ---")
hasil_penilaian_akhir_g5 = []

if not komponen_evaluasi_siap:
    print("Evaluasi tidak dapat dilanjutkan karena komponen penting tidak siap.")
elif not tasks_to_evaluate_final_g5:
    print("Tidak ada tugas untuk dievaluasi (data parsing dari file .txt gagal atau file kosong).")
else:
    print("Memulai Proses Evaluasi Jawaban dari file input...")
    for i_task, task_item_eval in enumerate(tasks_to_evaluate_final_g5):
        print(f"\nMengevaluasi Tugas ID: {task_item_eval.get('id_soal', f'Index_{i_task+1}')}")

        soal_eval_g5 = task_item_eval.get("pertanyaan", "")
        jawaban_siswa_eval_g5 = task_item_eval.get("jawaban_siswa", "")
        kata_kunci_eval_g5 = task_item_eval.get("kata_kunci", [])

        print(f"  Soal (awal): {soal_eval_g5[:80]}...") # Tampilkan 80 karakter pertama soal

        skor_final_eval_val = 0.0
        if not jawaban_siswa_eval_g5.strip():
            print("  Peringatan: Jawaban siswa untuk tugas ini kosong. Skor diatur ke 0.")
        else:
            # Panggil fungsi evaluate_answer_g5
            # Model untuk cosine sim adalah model besar pra-latih
            # Model untuk input LSTM adalah model fine-tuned (ft_model_global)
            skor_final_eval_val = evaluate_answer_g5(
                jawaban_siswa_text_param=jawaban_siswa_eval_g5,
                soal_text_param=soal_eval_g5,
                kata_kunci_list_param=kata_kunci_eval_g5,
                embedding_model_for_cosine_param=embedding_model_large_vocab_global,
                mean_vector_for_cosine_param=mean_vector_for_large_vocab_global,
                embedding_model_for_lstm_input=ft_model_global,
                mean_vector_for_lstm_input=mean_fasttext_vector_global,
                lstm_model_param=lstm_manual_model_instance_global
                # konteks_tfidf_dokumen bisa ditambahkan jika ada corpus yang lebih besar untuk IDF
            )

        hasil_penilaian_akhir_g5.append({
            "id_soal": task_item_eval.get("id_soal", f"Index_{i_task+1}"),
            "soal": soal_eval_g5,
            "jawaban_siswa_input": jawaban_siswa_eval_g5,
            "kata_kunci_input": kata_kunci_eval_g5,
            "skor_total_numerik": skor_final_eval_val,
            "skor_total_persen": f"{skor_final_eval_val * 100:.2f}"
        })
        print(f"  Skor Total Gabungan untuk Tugas '{task_item_eval.get('id_soal', '')}': {skor_final_eval_val * 100:.2f}/100")

    # --- Cetak dan Simpan Hasil Penilaian Akhir ---
    print("\n\n===== HASIL PENILAIAN AKHIR (dari input_soal.txt) =====")
    if not hasil_penilaian_akhir_g5:
        print("Tidak ada hasil penilaian untuk ditampilkan.")
    else:
        try:
            # Menggunakan Pandas untuk tampilan tabel jika tersedia dan diizinkan
            df_hasil_g5 = pd.DataFrame(hasil_penilaian_akhir_g5)
            # Tampilkan kolom yang relevan, sesuaikan dengan permintaan output Anda
            print(df_hasil_g5[['id_soal', 'soal', 'skor_total_persen']].to_string(index=False))
        except Exception as e:
            print(f"Error membuat DataFrame dengan Pandas: {e}. Mencetak hasil secara manual:")
            for hasil_item_print in hasil_penilaian_akhir_g5:
                print(f"\nID Tugas: {hasil_item_print['id_soal']}")
                print(f"Soal: {hasil_item_print['soal']}")
                print(f"Kata Kunci: {', '.join(hasil_item_print['kata_kunci_input']) if hasil_item_print['kata_kunci_input'] else '-'}")
                print(f"Skor Total: {hasil_item_print['skor_total_persen']}/100")
                print("-" * 40)

    # Path dari Grup 1: hasil_evaluasi_akhir_file_global
    if hasil_penilaian_akhir_g5: # Hanya simpan jika ada hasil
        try:
            with open(hasil_evaluasi_akhir_file_global, 'w', encoding='utf-8') as f_out_eval:
               json.dump(hasil_penilaian_akhir_g5, f_out_eval, indent=2, ensure_ascii=False)
            print(f"\nHasil penilaian akhir juga disimpan dalam format JSON di: {hasil_evaluasi_akhir_file_global}")
        except Exception as e:
            print(f"Gagal menyimpan hasil penilaian akhir ke JSON: {e}")

print("\n--- Grup 5 Selesai ---")